# =============================================================
# Federated Learning-Based Intrusion Detection System
# Complete Implementation: Centralized + FedAvg + FCWTL + Ring FL + Gossip FL
# With Gaussian and Label Flipping Attacks
# Datasets: CICIoMT2024, cicids2017, wustl2020
# =============================================================

In [1]:
import warnings, json, time, random, os
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix, log_loss)
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Set style
sns.set_style("whitegrid")

## CONFIGURATION

In [ ]:
# --- paths ---------------------------------------------------------------
DATA_PATHS = {
    "ciciomt2024": "/kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv",
    "cicids2017":  "/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv",
    "wustl2020":   "/kaggle/input/wustlehms2020/wustl-ehms-2020.csv",
}


RANDOM_SEED = 42
OUTPUT_ROOT = Path("fl_ids_results")
OUTPUT_ROOT.mkdir(exist_ok=True)

## 1. DATA LOADING AND PREPROCESSING

### 1‑A.  CICIoMT‑2024 

In [2]:
def load_and_preprocess_ciciomt2024(sample=None, seed=42):
    """Load and preprocess CICIoMT2024 dataset"""
    path = DATA_PATHS["ciciomt2024"]
    print(f"Loading CICIDS‑2024 from {path}")
    df = pd.read_csv(path, low_memory=False)
    
    if sample:
        df = df.sample(sample, random_state=seed)
        print(f"Sampled {sample} rows")
    
    print(f"Original shape: {df.shape}")
    
    # Remove potential data leakage columns
    leakage_cols = ["Attack Category", "SrcAddr", "DstAddr", "SrcMac", "DstMac", "Dir", "Flgs"]
    df = df.drop(columns=[c for c in leakage_cols if c in df.columns])
    
    # Remove constant columns
    constant_cols = df.columns[df.nunique() == 1]
    if len(constant_cols) > 0:
        df = df.drop(columns=constant_cols)
        print(f"Removed {len(constant_cols)} constant columns")
    
    # Prepare features and target
    y = df["Label"].astype(int).values
    X = df.drop(columns=["Label"])
    
    print(f"Final shape: {X.shape}, Target distribution: {np.bincount(y)}")
    
    # Identify numeric and categorical columns
    num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
    cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
    
    # Convert numeric-looking categorical columns
    for col in cat_cols.copy():
        if X[col].str.isnumeric().all():
            X[col] = X[col].astype(int)
            num_cols.append(col)
            cat_cols.remove(col)
    
    print(f"Numeric columns: {len(num_cols)}, Categorical columns: {len(cat_cols)}")
    
    # Create preprocessing pipeline
    preprocessor = ColumnTransformer([
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ])
    
    # Train-test split
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=seed
    )
    
    # Apply preprocessing
    X_train = preprocessor.fit_transform(X_train_raw)
    X_test = preprocessor.transform(X_test_raw)
    
    print(f"Preprocessed - Train: {X_train.shape}, Test: {X_test.shape}")
    
    return X_train, y_train, X_test, y_test, preprocessor


### 1‑B.  CICIDS‑2017 

In [3]:
def load_and_preprocess_cicids2017(sample=None, seed: int = 42):
    path = DATA_PATHS["cicids2017"]
    print(f"Loading CICIDS‑2017 from {path}")
    df = pd.read_csv(path, low_memory=False)

    if sample:
        df = df.sample(sample, random_state=seed)
        print(f"Sampled {sample} rows")

    df["Label"] = (df["Attack Type"] != "Normal Traffic").astype(int)
    y = df["Label"].values
    X = df.drop(columns=["Label", "Attack Type"]).astype(float)

    X_tr_raw, X_te_raw, y_tr, y_te = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=seed)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_tr_raw)
    X_test  = scaler.transform(X_te_raw)
    print(f"Pre‑processed CICIDS – Train {X_train.shape} | Test {X_test.shape}")
    return X_train, y_tr, X_test, y_te, scaler

### 1‑C.  WUSTL‑EHMS‑2020

In [ ]:
def load_and_preprocess_wustl2020(sample=None, seed: int = 42):
    path = DATA_PATHS["wustl2020"]
    print(f"Loading WUSTL‑EHMS‑2020 from {path}")
    df = pd.read_csv(path, low_memory=False)

    if sample:
        df = df.sample(sample, random_state=seed)
        print(f"Sampled {sample} rows")

    leaks = ["SrcAddr", "DstAddr", "SrcMac", "DstMac", "Dir", "Flgs"]
    df = df.drop(columns=[c for c in leaks if c in df.columns])
    df = df.drop(columns=df.columns[df.nunique() == 1])   

    y = df["Label"].astype(int).values
    X = df.drop(columns=["Label"])

    num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
    cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
    for col in cat_cols.copy():
        if X[col].str.isnumeric().all():
            X[col] = X[col].astype(int)
            num_cols.append(col); cat_cols.remove(col)

    print(f"Numeric {len(num_cols)} | Categorical {len(cat_cols)}")

    pre = ColumnTransformer([
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ])

    X_tr_raw, X_te_raw, y_tr, y_te = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=seed)

    X_train = pre.fit_transform(X_tr_raw)
    X_test  = pre.transform(X_te_raw)
    print(f"Pre‑processed WUSTL‑2020 – Train {X_train.shape} | Test {X_test.shape}")
    return X_train, y_tr, X_test, y_te, pre

### loader registry 

In [6]:
def load_dataset(name, sample=None, seed=42):
    if name == "ciciomt2024":
        return load_and_preprocess_ciciomt2024(sample, seed)      
    elif name == "cicids2017":
        return load_and_preprocess_cicids2017(sample, seed)     
    elif name == "wustl2020":
        return load_and_preprocess_wustl2020(sample, seed)       
    else:
        raise ValueError(f"Unknown dataset: {name}")

## 2. MODEL ARCHITECTURE

In [7]:
def create_model(input_dim):
    """Create DNN model as specified in requirements"""
    return Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

## 3. UTILITY FUNCTIONS

In [8]:
def dirichlet_split(n_clients, labels, alpha, seed=42):
    """Split data among clients using Dirichlet distribution (non-IID)"""
    np.random.seed(seed)
    
    # Get indices for each class
    class_indices = [np.where(labels == c)[0] for c in np.unique(labels)]
    client_indices = [[] for _ in range(n_clients)]
    
    for indices in class_indices:
        np.random.shuffle(indices)
        # Sample proportions from Dirichlet distribution
        proportions = np.random.dirichlet(alpha * np.ones(n_clients))
        # Convert proportions to actual counts
        proportions = (proportions / proportions.sum() * len(indices)).astype(int)
        proportions[-1] = len(indices) - proportions[:-1].sum()  
        
        # Distribute indices to clients
        start_idx = 0
        for client_id, count in enumerate(proportions):
            client_indices[client_id].extend(indices[start_idx:start_idx + count])
            start_idx += count
    
    return [np.array(indices, dtype=np.int64) for indices in client_indices]

def get_weights(model):
    """Get model weights"""
    return [w.copy() for w in model.get_weights()]

def set_weights(model, weights):
    """Set model weights"""
    model.set_weights(weights)

def add_gaussian_noise(weights, sigma):
    """Add Gaussian noise to model weights"""
    return [w + np.random.normal(0, sigma, w.shape) for w in weights]

def flip_labels(labels):
    """Flip binary labels for label flipping attack"""
    return 1 - labels

## 4. EVALUATION FUNCTIONS

In [9]:
def evaluate_model(model, X, y, batch_size=1024):
    """Evaluate model and return comprehensive metrics"""
    y_prob = model.predict(X, batch_size=batch_size, verbose=0).ravel()
    y_pred = (y_prob > 0.5).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    
    return {
        'accuracy': accuracy_score(y, y_pred),
        'precision': precision_score(y, y_pred, zero_division=0),
        'recall_tpr': recall_score(y, y_pred, zero_division=0),
        'f1_score': f1_score(y, y_pred, zero_division=0),
        'fpr': fp / (fp + tn) if (fp + tn) > 0 else 0.0,
        'roc_auc': roc_auc_score(y, y_prob),
        'test_loss': log_loss(y, y_prob),
        'tp': int(tp), 'tn': int(tn), 'fp': int(fp), 'fn': int(fn)
    }

def calculate_client_variance(client_weights, X_test, y_test):
    """Calculate variance in client performance"""
    accuracies = []
    input_dim = X_test.shape[1]
    
    for weights in client_weights:
        temp_model = create_model(input_dim)
        temp_model.compile(optimizer='adam', loss='binary_crossentropy')
        set_weights(temp_model, weights)
        
        y_pred = (temp_model.predict(X_test, batch_size=1024, verbose=0).ravel() > 0.5).astype(int)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        # Clean up
        tf.keras.backend.clear_session()
        del temp_model
    
    return float(np.var(accuracies))

## 5. CENTRALIZED LEARNING BASELINE

In [10]:
def run_centralized_learning(dataset, 
                            epochs=20, batch_size=256, output_dir="centralized"):
    """Run centralized learning baseline"""
    print("\n" + "="*60)
    print("RUNNING CENTRALIZED LEARNING BASELINE")
    print("="*60)

    X_train, y_train, X_test, y_test, _ = load_dataset(dataset, sample=None)
    
    # Create output directory
    out_dir = OUTPUT_ROOT / output_dir
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # Create and compile model
    model = create_model(X_train.shape[1])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Train model
    start_time = time.time()
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
        verbose=1
    )
    
    training_time = time.time() - start_time
    epochs_run = len(history.history['loss'])
    
    # Evaluate model
    metrics = evaluate_model(model, X_test, y_test)
    metrics.update({
        'topology': 'centralized',
        'attack_type': 'none',
        'attack_pct': 0.0,
        'training_time_sec': training_time,
        'epochs_run': epochs_run,
        'convergence_rounds': epochs_run
    })
    
    # Save results
    results_df = pd.DataFrame([metrics])
    results_df.to_csv(out_dir / "metrics.csv", index=False)
    
    # Save model
    model.save(out_dir / "model.h5")
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Centralized Learning - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Centralized Learning - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(out_dir / "training_history.png", dpi=150, bbox_inches='tight')
    plt.close()
    
    # Plot confusion matrix
    cm = np.array([[metrics['tn'], metrics['fp']], [metrics['fn'], metrics['tp']]])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Attack'], 
                yticklabels=['Normal', 'Attack'])
    plt.title('Centralized Learning - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(out_dir / "confusion_matrix.png", dpi=150, bbox_inches='tight')
    plt.close()
    
    # Save configuration
    config = {
        'dataset': 'CICIoMT2024',
        'topology': 'centralized',
        'attack_type': 'none',
        'attack_pct': 0.0,
        'epochs': epochs,
        'batch_size': batch_size,
        'final_accuracy': metrics['accuracy'],
        'final_f1': metrics['f1_score']
    }
    
    with open(out_dir / "config.json", 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\nCentralized Learning Results:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"F1-Score: {metrics['f1_score']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall_tpr']:.4f}")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Results saved to: {out_dir}")
    
    return metrics

## 6. FEDERATED TOPOLOGY IMPLEMENTATIONS

In [12]:
def fedavg_round(global_weights, X_train, y_train, client_indices, malicious_clients,
                attack_type, sigma, local_epochs, batch_size):
    """Execute one round of FedAvg"""
    n_clients = len(client_indices)
    local_weights = []
    local_sizes = []
    
    for client_id in range(n_clients):
        # Create local model
        local_model = create_model(X_train.shape[1])
        local_model.compile(optimizer='adam', loss='binary_crossentropy')
        set_weights(local_model, global_weights)
        
        # Get client data
        client_X = X_train[client_indices[client_id]]
        client_y = y_train[client_indices[client_id]]
        
        # Apply label flipping attack
        if client_id in malicious_clients and attack_type == "labelflip":
            client_y = flip_labels(client_y)
        
        # Local training
        local_model.fit(client_X, client_y, epochs=local_epochs, 
                       batch_size=batch_size, verbose=0)
        
        # Get updated weights
        weights = get_weights(local_model)
        
        # Apply Gaussian attack
        if client_id in malicious_clients and attack_type == "gaussian":
            weights = add_gaussian_noise(weights, sigma)
        
        local_weights.append(weights)
        local_sizes.append(len(client_X))
        
        # Clean up
        tf.keras.backend.clear_session()
        del local_model
    
    # Aggregate weights (weighted average)
    total_size = sum(local_sizes)
    aggregated_weights = []
    
    for layer_idx in range(len(local_weights[0])):
        layer_weights = sum(weights[layer_idx] * size / total_size 
                           for weights, size in zip(local_weights, local_sizes))
        aggregated_weights.append(layer_weights)
    
    # Communication cost: (n_clients + 1) * model_size
    comm_cost = (n_clients + 1) * sum(w.nbytes for w in global_weights)
    
    return aggregated_weights, local_weights, comm_cost

def fcwtl_round(global_weights, X_train, y_train, client_indices, malicious_clients,
               attack_type, sigma, local_epochs, batch_size, n_clients):
    """Execute one round of FCWTL (Federated Cyclic Weight Transfer Learning)"""
    current_weights = global_weights.copy()
    
    for client_id in range(n_clients):
        # Create local model
        local_model = create_model(X_train.shape[1])
        local_model.compile(optimizer='adam', loss='binary_crossentropy')
        set_weights(local_model, current_weights)
        
        # Get client data
        client_X = X_train[client_indices[client_id]]
        client_y = y_train[client_indices[client_id]]
        
        # Apply label flipping attack
        if client_id in malicious_clients and attack_type == "labelflip":
            client_y = flip_labels(client_y)
        
        # Local training
        local_model.fit(client_X, client_y, epochs=local_epochs, 
                       batch_size=batch_size, verbose=0)
        
        # Get updated weights
        current_weights = get_weights(local_model)
        
        # Apply Gaussian attack
        if client_id in malicious_clients and attack_type == "gaussian":
            current_weights = add_gaussian_noise(current_weights, sigma)
        
        # Clean up
        tf.keras.backend.clear_session()
        del local_model
    
    # In FCWTL, we only have the final weights
    client_weights = [current_weights]
    comm_cost = (n_clients + 1) * sum(w.nbytes for w in global_weights)
    
    return current_weights, client_weights, comm_cost

def ring_fl_round(global_weights, X_train, y_train, client_indices, malicious_clients,
                 attack_type, sigma, local_epochs, batch_size, n_clients):
    """Execute one round of Ring FL"""
    # Initialize ring with global weights for each client
    ring_weights = [global_weights.copy() for _ in range(n_clients)]
    
    # Ring communication: each client trains and passes to next
    for step in range(n_clients):
        sender = step
        receiver = (step + 1) % n_clients
        
        # Create local model for sender
        local_model = create_model(X_train.shape[1])
        local_model.compile(optimizer='adam', loss='binary_crossentropy')
        set_weights(local_model, ring_weights[sender])
        
        # Get sender data
        client_X = X_train[client_indices[sender]]
        client_y = y_train[client_indices[sender]]
        
        # Apply label flipping attack
        if sender in malicious_clients and attack_type == "labelflip":
            client_y = flip_labels(client_y)
        
        # Local training
        local_model.fit(client_X, client_y, epochs=local_epochs, 
                       batch_size=batch_size, verbose=0)
        
        # Get updated weights
        updated_weights = get_weights(local_model)
        
        # Apply Gaussian attack
        if sender in malicious_clients and attack_type == "gaussian":
            updated_weights = add_gaussian_noise(updated_weights, sigma)
        
        # Pass to receiver
        ring_weights[receiver] = updated_weights
        
        # Clean up
        tf.keras.backend.clear_session()
        del local_model
    
    # Final global model is the one that completed the ring
    final_weights = ring_weights[0]
    comm_cost = (n_clients + 1) * sum(w.nbytes for w in global_weights)
    
    return final_weights, ring_weights, comm_cost

def gossip_fl_round(global_weights, X_train, y_train, client_indices, malicious_clients,
                   attack_type, sigma, local_epochs, batch_size, n_clients):
    """Execute one round of Gossip FL"""
    local_weights = []
    local_sizes = []
    
    # Local training phase
    for client_id in range(n_clients):
        # Create local model
        local_model = create_model(X_train.shape[1])
        local_model.compile(optimizer='adam', loss='binary_crossentropy')
        set_weights(local_model, global_weights)
        
        # Get client data
        client_X = X_train[client_indices[client_id]]
        client_y = y_train[client_indices[client_id]]
        
        # Apply label flipping attack
        if client_id in malicious_clients and attack_type == "labelflip":
            client_y = flip_labels(client_y)
        
        # Local training
        local_model.fit(client_X, client_y, epochs=local_epochs, 
                       batch_size=batch_size, verbose=0)
        
        # Get updated weights
        weights = get_weights(local_model)
        
        # Apply Gaussian attack
        if client_id in malicious_clients and attack_type == "gaussian":
            weights = add_gaussian_noise(weights, sigma)
        
        local_weights.append(weights)
        local_sizes.append(len(client_X))
        
        # Clean up
        tf.keras.backend.clear_session()
        del local_model
    
    # Gossip communication: random pairwise averaging
    shuffled_indices = random.sample(range(n_clients), n_clients)
    gossip_comm_cost = 0
    
    for i in range(0, len(shuffled_indices) - 1, 2):
        client_a = shuffled_indices[i]
        client_b = shuffled_indices[i + 1]
        
        # Average weights between pair
        averaged_weights = []
        for layer_idx in range(len(local_weights[0])):
            avg_layer = (local_weights[client_a][layer_idx] + 
                        local_weights[client_b][layer_idx]) / 2
            averaged_weights.append(avg_layer)
        
        # Both clients get the averaged weights
        local_weights[client_a] = averaged_weights
        local_weights[client_b] = averaged_weights
        
        gossip_comm_cost += 2 * sum(w.nbytes for w in averaged_weights)
    
    # Final aggregation (weighted average)
    total_size = sum(local_sizes)
    final_weights = []
    
    for layer_idx in range(len(local_weights[0])):
        layer_weights = sum(weights[layer_idx] * size / total_size 
                           for weights, size in zip(local_weights, local_sizes))
        final_weights.append(layer_weights)
    
    # Total communication cost
    comm_cost = gossip_comm_cost + (n_clients + 1) * sum(w.nbytes for w in global_weights)
    
    return final_weights, local_weights, comm_cost

# =============================================================
# 8. VISUALIZATION FUNCTIONS
# =============================================================
def create_federated_visualizations(results_df, best_cm, worst_cm, out_dir, 
                                   topology, attack_type, attack_pct):
    """Create plots (accuracy / loss, confusion matrices, comm. cost)"""

    # ── 1. Accuracy & loss curves ────────────────────────────────
    plt.figure(figsize=(15, 4))
    plt.subplot(1, 2, 1)
    plt.plot(results_df['round'], results_df['accuracy'], label='Accuracy', lw=2)
    plt.title(f'{topology.upper()} – {attack_type} {int(attack_pct*100)}% | Accuracy')
    plt.xlabel('Round'); plt.ylabel('Accuracy'); plt.grid(alpha=.3); plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(results_df['round'], results_df['test_loss'], label='Loss', color='tab:red', lw=2)
    plt.title(f'{topology.upper()} – {attack_type} {int(attack_pct*100)}% | Loss')
    plt.xlabel('Round'); plt.ylabel('Binary‑Cross‑Entropy'); plt.grid(alpha=.3); plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / "acc_loss_curves.png", dpi=150, bbox_inches='tight')
    plt.close()

    # ── 2. Best / worst confusion matrices ───────────────────────
    for tag, cm in [('best', best_cm), ('worst', worst_cm)]:
        if cm is None: continue
        plt.figure(figsize=(4, 3.5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Normal', 'Attack'],
                    yticklabels=['Normal', 'Attack'])
        plt.title(f'{topology.upper()} – {tag.capitalize()} Confusion')
        plt.ylabel('True'); plt.xlabel('Predicted')
        plt.tight_layout()
        plt.savefig(out_dir / f'conf_matrix_{tag}.png', dpi=150, bbox_inches='tight')
        plt.close()

    # ── 3. Communication cost over time ──────────────────────────
    plt.figure(figsize=(7, 4))
    plt.plot(results_df['round'], results_df['total_comm_bytes']/1e6,
             label='Cumulative MB', lw=2, color='tab:green')
    plt.title(f'{topology.upper()} – Communication Over Rounds')
    plt.xlabel('Round'); plt.ylabel('Cumulative MB'); plt.grid(alpha=.3); plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / "communication_cost.png", dpi=150, bbox_inches='tight')
    plt.close()


## 7. FEDERATED LEARNING IMPLEMENTATIONS

In [13]:
def run_federated_learning(topology, dataset,
                          attack_type="none", attack_pct=0.0, sigma=2,
                          n_clients=15, rounds=30, local_epochs=1, 
                          batch_size=256, dir_alpha=0.3, seed=42,
                          output_dir=None):
    """Run federated learning with specified topology and attack"""
    X_train, y_train, X_test, y_test, _ = load_dataset(dataset, sample=None)
    if output_dir is None:
        output_dir = f"{topology}_{attack_type}_{attack_pct}"
    
    print(f"\n" + "="*60)
    print(f"RUNNING {topology.upper()} - Attack: {attack_type} ({attack_pct*100:.0f}%)")
    print("="*60)
    
    # Set seeds
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    
    # Create output directory
    out_dir = OUTPUT_ROOT / output_dir
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # Split data among clients (non-IID)
    client_indices = dirichlet_split(n_clients, y_train, dir_alpha, seed)
    
    # Initialize global model
    global_model = create_model(X_train.shape[1])
    global_model.compile(optimizer='adam', loss='binary_crossentropy')
    global_weights = get_weights(global_model)
    
    # Calculate communication cost per round
    bytes_per_round = sum(w.nbytes for w in global_weights)
    total_comm_bytes = 0
    
    # Training logs
    training_logs = []
    best_accuracy = -1
    worst_accuracy = 2
    best_cm = None
    worst_cm = None
    
    # Main federated learning loop
    for round_num in range(1, rounds + 1):
        round_start_time = time.time()
        
        # Select malicious clients for this round
        malicious_clients = []
        if attack_type != "none" and attack_pct > 0:
            n_malicious = int(attack_pct * n_clients)
            malicious_clients = random.sample(range(n_clients), n_malicious)
        
        # Execute federated learning topology
        if topology == "fedavg":
            global_weights, client_weights, comm_cost = fedavg_round(
                global_weights, X_train, y_train, client_indices, malicious_clients,
                attack_type, sigma, local_epochs, batch_size
            )
        elif topology == "fcwtl":
            global_weights, client_weights, comm_cost = fcwtl_round(
                global_weights, X_train, y_train, client_indices, malicious_clients,
                attack_type, sigma, local_epochs, batch_size, n_clients
            )
        elif topology == "ring":
            global_weights, client_weights, comm_cost = ring_fl_round(
                global_weights, X_train, y_train, client_indices, malicious_clients,
                attack_type, sigma, local_epochs, batch_size, n_clients
            )
        elif topology == "gossip":
            global_weights, client_weights, comm_cost = gossip_fl_round(
                global_weights, X_train, y_train, client_indices, malicious_clients,
                attack_type, sigma, local_epochs, batch_size, n_clients
            )
        else:
            raise ValueError(f"Unknown topology: {topology}")
        
        total_comm_bytes += comm_cost
        
        # Update global model
        set_weights(global_model, global_weights)
        
        # Evaluate global model
        metrics = evaluate_model(global_model, X_test, y_test)
        client_var = calculate_client_variance(client_weights, X_test, y_test)
        
        # Update training logs
        round_metrics = metrics.copy()
        round_metrics.update({
            'round': round_num,
            'topology': topology,
            'attack_type': attack_type,
            'attack_pct': attack_pct,
            'sigma': sigma if attack_type == 'gaussian' else None,
            'malicious_clients': ';'.join(map(str, malicious_clients)),
            'client_variance': client_var,
            'comm_bytes_round': comm_cost,
            'total_comm_bytes': total_comm_bytes,
            'round_time_sec': time.time() - round_start_time
        })
        training_logs.append(round_metrics)
        
        # Track best and worst performance
        if metrics['accuracy'] > best_accuracy:
            best_accuracy = metrics['accuracy']
            best_cm = np.array([[metrics['tn'], metrics['fp']], 
                               [metrics['fn'], metrics['tp']]])
        
        if metrics['accuracy'] < worst_accuracy:
            worst_accuracy = metrics['accuracy']
            worst_cm = np.array([[metrics['tn'], metrics['fp']], 
                                [metrics['fn'], metrics['tp']]])
        
        # Print progress
        print(f"Round {round_num:2d}/{rounds} | "
              f"Acc: {metrics['accuracy']:.4f} | "
              f"F1: {metrics['f1_score']:.4f} | "
              f"Var: {client_var:.4f} | "
              f"Malicious: {len(malicious_clients)}")
    
    # Save results
    results_df = pd.DataFrame(training_logs)
    results_df.to_csv(out_dir / "metrics.csv", index=False)
    
    # Save final model
    global_model.save(out_dir / "final_model.h5")
    
    # Create visualizations
    create_federated_visualizations(results_df, best_cm, worst_cm, out_dir, topology, attack_type, attack_pct)
    
    # Save configuration
    config = {
        'dataset': 'CICIoMT2024',
        'topology': topology,
        'attack_type': attack_type,
        'attack_pct': attack_pct,
        'sigma': sigma,
        'n_clients': n_clients,
        'rounds': rounds,
        'local_epochs': local_epochs,
        'batch_size': batch_size,
        'dir_alpha': dir_alpha,
        'bytes_per_round': bytes_per_round,
        'total_comm_bytes': total_comm_bytes,
        'final_accuracy': results_df.iloc[-1]['accuracy'],
        'final_f1': results_df.iloc[-1]['f1_score'],
        'best_accuracy': best_accuracy,
        'worst_accuracy': worst_accuracy
    }
    
    with open(out_dir / "config.json", 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\n{topology.upper()} Results:")
    print(f"Final Accuracy: {results_df.iloc[-1]['accuracy']:.4f}")
    print(f"Final F1-Score: {results_df.iloc[-1]['f1_score']:.4f}")
    print(f"Best Accuracy: {best_accuracy:.4f}")
    print(f"Total Communication: {total_comm_bytes / (1024*1024):.2f} MB")
    print(f"Results saved to: {out_dir}")
    
    return results_df.iloc[-1].to_dict()

# Centralized 


#### Configuration
###### dataset in ["ciciomt2024", "cicids2017", "wustl2020"]

In [2]:
run_centralized_learning("ciciomt2024",
                         epochs=20, batch_size=256,
                         output_dir="ciciomt2024_centralized_clean")


RUNNING CENTRALIZED LEARNING BASELINE
Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)


I0000 00:00:1753282082.430279   72314 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/20


I0000 00:00:1753282087.186941   72367 service.cc:148] XLA service 0x7e13e000b080 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753282087.186997   72367 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753282087.419599   72367 cuda_dnn.cc:529] Loaded cuDNN version 90300


20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8435 - loss: 0.6414

I0000 00:00:1753282088.943551   72367 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.8586 - loss: 0.5579 - val_accuracy: 0.9211 - val_loss: 0.2736
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9128 - loss: 0.2870 - val_accuracy: 0.9364 - val_loss: 0.2305
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9215 - loss: 0.2495 - val_accuracy: 0.9380 - val_loss: 0.2172
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9236 - loss: 0.2130 - val_accuracy: 0.9403 - val_loss: 0.2179
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9360 - loss: 0.1539 - val_accuracy: 0.9395 - val_loss: 0.2215
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9830 - loss: 0.0605 - val_accuracy: 0.9234 - val_loss: 0.2590

Centralized Learning Results:
Accuracy: 0.9363
F1-Score: 0.6719
Precision: 0.9467
Recall: 0.5208
Training Time: 11.44 seconds
Results saved to: fl_ids_results/ciciomt2024_centralized_clean


{'accuracy': 0.9362745098039216,
 'precision': 0.9466666666666667,
 'recall_tpr': 0.5207823960880196,
 'f1_score': 0.6719242902208202,
 'fpr': 0.004203152364273205,
 'roc_auc': 0.8693357426382746,
 'test_loss': 0.21831430468245025,
 'tp': 213,
 'tn': 2843,
 'fp': 12,
 'fn': 196,
 'topology': 'centralized',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'training_time_sec': 11.438568592071533,
 'epochs_run': 6,
 'convergence_rounds': 6}

In [3]:
run_centralized_learning("cicids2017",
                         epochs=20, batch_size=256,
                         output_dir="cicids2017_centralized_clean")


RUNNING CENTRALIZED LEARNING BASELINE
Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
7090/7090 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9779 - loss: 0.0494 - val_accuracy: 0.9807 - val_loss: 0.0416
Epoch 3/20
7090/7090 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9793 - loss: 0.0456 - val_accuracy: 0.9825 - val_loss: 0.0391
Epoch 4/20
7090/7090 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9805 - loss: 0.0427 - val_accuracy: 0.9816 - val_loss: 0.0381
Epoch 5/20
7090/7090 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9811 - loss: 0.0415 - val_accuracy: 0.9837 - val_loss: 0.0355
Epoch 6/20
7090/7090 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9813 - loss: 0.0412 - val_accuracy: 0.9860 - val_loss: 0.0329
Epoch 7/20
7090/7090 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9821 - loss: 0.0393 - val_accuracy: 0.9833 - val_loss: 0.0357
Epoch 8/20
7090/7090 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9822 - loss: 0.0401

{'accuracy': 0.9856729432253432,
 'precision': 0.9613799474170397,
 'recall_tpr': 0.9534643347936903,
 'f1_score': 0.9574057802650124,
 'fpr': 0.007782593338615601,
 'roc_auc': 0.9988574712845697,
 'test_loss': 0.03271208142251364,
 'tp': 81177,
 'tn': 415751,
 'fp': 3261,
 'fn': 3962,
 'topology': 'centralized',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'training_time_sec': 172.99662613868713,
 'epochs_run': 9,
 'convergence_rounds': 9}

In [4]:
run_centralized_learning("wustl2020",
                         epochs=20, batch_size=256,
                         output_dir="wustl2020_centralized_clean")


RUNNING CENTRALIZED LEARNING BASELINE
Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)
Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.8374 - loss: 0.5547 - val_accuracy: 0.9219 - val_loss: 0.2686
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9180 - loss: 0.2837 - val_accuracy: 0.9372 - val_loss: 0.2255
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9239 - loss: 0.2446 - val_accuracy: 0.9387 - val_loss: 0.2163
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9230 - loss: 0.2184 - val_accuracy: 0.9380 - val_loss: 0.2146
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9351 - loss: 0.1601 - val_accuracy: 0.9273 - val_loss: 0.2318
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9752 - loss: 0.0662 - val_accuracy: 0.9257 - val_loss: 0.2316
Epoch 7/20
46/46 ━━━━━━━━━

{'accuracy': 0.9368872549019608,
 'precision': 0.9393939393939394,
 'recall_tpr': 0.530562347188264,
 'f1_score': 0.6781249999999999,
 'fpr': 0.004903677758318739,
 'roc_auc': 0.8758579937398037,
 'test_loss': 0.20951498107535282,
 'tp': 217,
 'tn': 2841,
 'fp': 14,
 'fn': 192,
 'topology': 'centralized',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'training_time_sec': 10.330350875854492,
 'epochs_run': 7,
 'convergence_rounds': 7}

# Federated


#### Configuration
#####     datasets = ["ciciomt2024", "cicids2017", "wustl2020"]
#####     topologies = ["fedavg", "fcwtl", "ring", "gossip"]
#####     attack_configs = [
#####         {"type": "none", "pct": 0.0},
#####         {"type": "gaussian", "pct": 0.2, "sigma": 0.1},
#####        {"type": "gaussian", "pct": 0.4, "sigma": 0.1},
#####        {"type": "labelflip", "pct": 0.2},
#####        {"type": "labelflip", "pct": 0.4}
#####     ]

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING FEDAVG - Attack: gaussian (20%)


I0000 00:00:1753030730.070333   22014 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753030733.224196   22069 service.cc:148] XLA service 0x79c63800be50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753030733.224239   22069 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753030733.432852   22069 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753030734.807945   22069 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.5432 | F1: 0.2787 | Var: 0.1058 | Malicious: 3
Round  2/30 | Acc: 0.7760 | F1: 0.2313 | Var: 0.0108 | Malicious: 3
Round  3/30 | Acc: 0.7353 | F1: 0.1970 | Var: 0.0560 | Malicious: 3
Round  4/30 | Acc: 0.7344 | F1: 0.2224 | Var: 0.0312 | Malicious: 3
Round  5/30 | Acc: 0.7537 | F1: 0.2328 | Var: 0.0235 | Malicious: 3
Round  6/30 | Acc: 0.5631 | F1: 0.1729 | Var: 0.0066 | Malicious: 3
Round  7/30 | Acc: 0.6115 | F1: 0.2114 | Var: 0.0270 | Malicious: 3
Round  8/30 | Acc: 0.7013 | F1: 0.2181 | Var: 0.0288 | Malicious: 3
Round  9/30 | Acc: 0.8263 | F1: 0.1818 | Var: 0.0231 | Malicious: 3
Round 10/30 | Acc: 0.8615 | F1: 0.0776 | Var: 0.0565 | Malicious: 3
Round 11/30 | Acc: 0.7619 | F1: 0.3809 | Var: 0.0480 | Malicious: 3
Round 12/30 | Acc: 0.7184 | F1: 0.0981 | Var: 0.0326 | Malicious: 3
Round 13/30 | Acc: 0.3729 | F1: 0.1446 | Var: 0.0302 | Malicious: 3
Round 14/30 | Acc: 0.3067 | F1: 0.2279 | Var: 0.0042 | Malicious: 3
Round 15/30 | Acc: 0.2684 | F1: 0.2077 | Var: 0.

{'accuracy': 0.7610294117647058,
 'precision': 0.12974051896207583,
 'recall_tpr': 0.15892420537897312,
 'f1_score': 0.14285714285714285,
 'fpr': 0.15271453590192643,
 'roc_auc': 0.5031048347385234,
 'test_loss': 3.8097612485788335,
 'tp': 65,
 'tn': 2419,
 'fp': 436,
 'fn': 344,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '2;6;3',
 'client_variance': 0.021263800114272292,
 'comm_bytes_round': 215679104,
 'total_comm_bytes': 6362533568,
 'round_time_sec': 185.10578203201294}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING FCWTL - Attack: gaussian (40%)


I0000 00:00:1753035378.065629   44402 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753035381.282815   44455 service.cc:148] XLA service 0x78057c08da40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753035381.282866   44455 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753035381.497152   44455 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753035382.902119   44455 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8753 | F1: 0.0145 | Var: 0.0000 | Malicious: 6
Round  2/30 | Acc: 0.5855 | F1: 0.3086 | Var: 0.0000 | Malicious: 6
Round  3/30 | Acc: 0.4001 | F1: 0.2561 | Var: 0.0000 | Malicious: 6
Round  4/30 | Acc: 0.1455 | F1: 0.2199 | Var: 0.0000 | Malicious: 6
Round  5/30 | Acc: 0.5864 | F1: 0.2880 | Var: 0.0000 | Malicious: 6
Round  6/30 | Acc: 0.3217 | F1: 0.2328 | Var: 0.0000 | Malicious: 6
Round  7/30 | Acc: 0.4256 | F1: 0.2085 | Var: 0.0000 | Malicious: 6
Round  8/30 | Acc: 0.3186 | F1: 0.2191 | Var: 0.0000 | Malicious: 6
Round 10/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.1256 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.1278 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.1281 | F1: 0.2207 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.1250 | F1: 0.2222 | Var: 0.0000 | Malicious: 6
Round 15/30 | Acc: 0.1305 | F1: 0.2233 | Var: 0.0000 | Malicious: 6
Round 16/30 | Acc: 0.1798 | F1: 0.2252 | Var: 0.

{'accuracy': 0.2689950980392157,
 'precision': 0.1369812706573632,
 'recall_tpr': 0.9119804400977995,
 'f1_score': 0.23818646232439336,
 'fpr': 0.8231173380035026,
 'roc_auc': 0.5444315510471485,
 'test_loss': 11.653961727767907,
 'tp': 373,
 'tn': 505,
 'fp': 2350,
 'fn': 36,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '10;12;9;4;14;3',
 'client_variance': 0.0,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 4097902976,
 'round_time_sec': 118.62129974365234}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING RING - Attack: gaussian (40%)


I0000 00:00:1753038442.351544   55230 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753038445.591216   55284 service.cc:148] XLA service 0x7e76bc08f040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753038445.591262   55284 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753038445.805260   55284 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753038447.211327   55284 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8753 | F1: 0.0145 | Var: 0.0035 | Malicious: 6
Round  2/30 | Acc: 0.5763 | F1: 0.3054 | Var: 0.0072 | Malicious: 6
Round  3/30 | Acc: 0.3995 | F1: 0.2559 | Var: 0.0058 | Malicious: 6
Round  4/30 | Acc: 0.1455 | F1: 0.2199 | Var: 0.0034 | Malicious: 6
Round  5/30 | Acc: 0.5818 | F1: 0.2865 | Var: 0.0329 | Malicious: 6
Round  6/30 | Acc: 0.3223 | F1: 0.2330 | Var: 0.0070 | Malicious: 6
Round  7/30 | Acc: 0.4243 | F1: 0.2102 | Var: 0.0054 | Malicious: 6
Round  8/30 | Acc: 0.3162 | F1: 0.2185 | Var: 0.0030 | Malicious: 6
Round  9/30 | Acc: 0.1247 | F1: 0.2217 | Var: 0.0068 | Malicious: 6
Round 10/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.1256 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.1278 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.1284 | F1: 0.2208 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.1250 | F1: 0.2222 | Var: 0.0000 | Malicious: 6
Round 15/30 | Acc: 0.1314 | F1: 0.2235 | Var: 0.

{'accuracy': 0.2699142156862745,
 'precision': 0.1368653421633554,
 'recall_tpr': 0.9095354523227384,
 'f1_score': 0.23792772625519668,
 'fpr': 0.8217162872154116,
 'roc_auc': 0.5439095825536634,
 'test_loss': 11.639308800347253,
 'tp': 372,
 'tn': 509,
 'fp': 2346,
 'fn': 37,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.0003117941416762782,
 'comm_bytes_round': 215679104,
 'total_comm_bytes': 4744940288,
 'round_time_sec': 185.27572774887085}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING GOSSIP - Attack: gaussian (40%)


I0000 00:00:1753085233.318373      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753085237.126087      95 service.cc:148] XLA service 0x7a4d6c00a480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753085237.126661      95 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753085237.414020      95 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753085238.939393      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8759 | F1: 0.2541 | Var: 0.0590 | Malicious: 6
Round  3/30 | Acc: 0.1575 | F1: 0.2174 | Var: 0.0133 | Malicious: 6
Round  4/30 | Acc: 0.1667 | F1: 0.2299 | Var: 0.0192 | Malicious: 6
Round  5/30 | Acc: 0.2714 | F1: 0.2422 | Var: 0.0596 | Malicious: 6
Round  6/30 | Acc: 0.2739 | F1: 0.2370 | Var: 0.0157 | Malicious: 6
Round  7/30 | Acc: 0.1900 | F1: 0.2309 | Var: 0.0222 | Malicious: 6
Round  8/30 | Acc: 0.4118 | F1: 0.2264 | Var: 0.0130 | Malicious: 6
Round  9/30 | Acc: 0.2972 | F1: 0.2192 | Var: 0.0382 | Malicious: 6
Round 10/30 | Acc: 0.2607 | F1: 0.2034 | Var: 0.0061 | Malicious: 6
Round 11/30 | Acc: 0.2430 | F1: 0.2251 | Var: 0.0318 | Malicious: 6
Round 12/30 | Acc: 0.2013 | F1: 0.2006 | Var: 0.0031 | Malicious: 6
Round 13/30 | Acc: 0.1477 | F1: 0.1973 | Var: 0.0023 | Malicious: 6
Round 14/30 | Acc: 0.1924 | F1: 0.2079 | Var: 0.0211 | Malicious: 6
Round 15/30 | Acc: 0.1400 | F1: 0.2214 | Var: 0.0120 | Malicious: 6
Round 16/30 | Acc: 0.1373 | F1: 0.2156 | Var: 0.

{'accuracy': 0.1253063725490196,
 'precision': 0.1253063725490196,
 'recall_tpr': 1.0,
 'f1_score': 0.22270623468554315,
 'fpr': 1.0,
 'roc_auc': 0.5,
 'test_loss': 13.944702714529845,
 'tp': 409,
 'tn': 0,
 'fp': 2855,
 'fn': 0,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '12;7;10;5;8;4',
 'client_variance': 0.009787187052789522,
 'comm_bytes_round': 390918376,
 'total_comm_bytes': 11107473856,
 'round_time_sec': 182.78005266189575}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING GOSSIP - Attack: none (0%)


I0000 00:00:1753258119.864653      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753258123.893255      98 service.cc:148] XLA service 0x7a7a4800c290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753258123.894240      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753258124.218291      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753258125.842694      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8735 | F1: 0.0373 | Var: 0.1070 | Malicious: 0
Round  2/30 | Acc: 0.8768 | F1: 0.0337 | Var: 0.0003 | Malicious: 0
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 16/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.8746936274509803,
 'precision': 0.0,
 'recall_tpr': 0.0,
 'f1_score': 0.0,
 'fpr': 0.0,
 'roc_auc': 0.8192259108756995,
 'test_loss': 0.43206290422793087,
 'tp': 0,
 'tn': 2855,
 'fp': 0,
 'fn': 409,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 1.232595164407831e-32,
 'comm_bytes_round': 202199160,
 'total_comm_bytes': 6065974800,
 'round_time_sec': 192.68370652198792}

In [1]:
1

1

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING GOSSIP - Attack: none (0%)


I0000 00:00:1753175881.935339   22586 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753175885.138303   22637 service.cc:148] XLA service 0x7887e408c710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753175885.138379   22637 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753175885.343559   22637 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753175886.716257   22637 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8735 | F1: 0.0373 | Var: 0.1070 | Malicious: 0
Round  2/30 | Acc: 0.8768 | F1: 0.0337 | Var: 0.0003 | Malicious: 0
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 16/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.8746936274509803,
 'precision': 0.0,
 'recall_tpr': 0.0,
 'f1_score': 0.0,
 'fpr': 0.0,
 'roc_auc': 0.8192259108756995,
 'test_loss': 0.43206290422793087,
 'tp': 0,
 'tn': 2855,
 'fp': 0,
 'fn': 409,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 1.232595164407831e-32,
 'comm_bytes_round': 202199160,
 'total_comm_bytes': 6065974800,
 'round_time_sec': 185.1821084022522}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FEDAVG - Attack: gaussian (40%)


I0000 00:00:1753090249.154698   23978 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753090252.345818   24031 service.cc:148] XLA service 0x7cc75400b8c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753090252.345866   24031 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753090252.557515   24031 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753090253.942570   24031 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8597 | F1: 0.0172 | Var: 0.0981 | Malicious: 6
Round  2/30 | Acc: 0.7497 | F1: 0.0577 | Var: 0.0805 | Malicious: 6
Round  3/30 | Acc: 0.1829 | F1: 0.1438 | Var: 0.0464 | Malicious: 6
Round  4/30 | Acc: 0.2249 | F1: 0.2301 | Var: 0.0207 | Malicious: 6
Round  5/30 | Acc: 0.7209 | F1: 0.2899 | Var: 0.0307 | Malicious: 6
Round  6/30 | Acc: 0.3640 | F1: 0.1520 | Var: 0.0377 | Malicious: 6
Round  7/30 | Acc: 0.4537 | F1: 0.1432 | Var: 0.0273 | Malicious: 6
Round  8/30 | Acc: 0.3928 | F1: 0.1261 | Var: 0.0327 | Malicious: 6
Round  9/30 | Acc: 0.3569 | F1: 0.1294 | Var: 0.0095 | Malicious: 6
Round 10/30 | Acc: 0.4418 | F1: 0.0989 | Var: 0.0102 | Malicious: 6
Round 11/30 | Acc: 0.2341 | F1: 0.1427 | Var: 0.0251 | Malicious: 6
Round 12/30 | Acc: 0.3511 | F1: 0.1313 | Var: 0.0310 | Malicious: 6
Round 13/30 | Acc: 0.3977 | F1: 0.1247 | Var: 0.0302 | Malicious: 6
Round 14/30 | Acc: 0.4317 | F1: 0.1595 | Var: 0.0331 | Malicious: 6
Round 15/30 | Acc: 0.5490 | F1: 0.1111 | Var: 0.

{'accuracy': 0.8115808823529411,
 'precision': 0.17197452229299362,
 'recall_tpr': 0.13202933985330073,
 'f1_score': 0.14937759336099582,
 'fpr': 0.09106830122591944,
 'roc_auc': 0.5204805193136905,
 'test_loss': 3.00385024044297,
 'tp': 54,
 'tn': 2595,
 'fp': 260,
 'fn': 355,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.039930290232923236,
 'comm_bytes_round': 215679104,
 'total_comm_bytes': 6362533568,
 'round_time_sec': 184.11950588226318}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FCWTL - Attack: gaussian (40%)


I0000 00:00:1753094702.726609   44957 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753094706.126492   45010 service.cc:148] XLA service 0x7f8a5400a540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753094706.126534   45010 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753094706.344347   45010 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753094707.773782   45010 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8753 | F1: 0.0145 | Var: 0.0000 | Malicious: 6
Round  2/30 | Acc: 0.5855 | F1: 0.3086 | Var: 0.0000 | Malicious: 6
Round  3/30 | Acc: 0.4001 | F1: 0.2561 | Var: 0.0000 | Malicious: 6
Round  4/30 | Acc: 0.1455 | F1: 0.2199 | Var: 0.0000 | Malicious: 6
Round  5/30 | Acc: 0.5864 | F1: 0.2880 | Var: 0.0000 | Malicious: 6
Round  6/30 | Acc: 0.3217 | F1: 0.2328 | Var: 0.0000 | Malicious: 6
Round  7/30 | Acc: 0.4256 | F1: 0.2085 | Var: 0.0000 | Malicious: 6
Round  8/30 | Acc: 0.3186 | F1: 0.2191 | Var: 0.0000 | Malicious: 6
Round  9/30 | Acc: 0.1247 | F1: 0.2217 | Var: 0.0000 | Malicious: 6
Round 10/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.1256 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.1278 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.1281 | F1: 0.2207 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.1250 | F1: 0.2222 | Var: 0.0000 | Malicious: 6
Round 15/30 | Acc: 0.1305 | F1: 0.2233 | Var: 0.

{'accuracy': 0.2689950980392157,
 'precision': 0.1369812706573632,
 'recall_tpr': 0.9119804400977995,
 'f1_score': 0.23818646232439336,
 'fpr': 0.8231173380035026,
 'roc_auc': 0.5444315510471485,
 'test_loss': 11.653961727767907,
 'tp': 373,
 'tn': 505,
 'fp': 2350,
 'fn': 36,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '10;12;9;4;14;3',
 'client_variance': 0.0,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 4097902976,
 'round_time_sec': 120.9620475769043}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING RING - Attack: gaussian (40%)


I0000 00:00:1753099020.340140   55787 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753099023.624219   55842 service.cc:148] XLA service 0x799b80087f10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753099023.624277   55842 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753099023.838195   55842 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753099025.252882   55842 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8753 | F1: 0.0145 | Var: 0.0035 | Malicious: 6
Round  2/30 | Acc: 0.5763 | F1: 0.3054 | Var: 0.0072 | Malicious: 6
Round  3/30 | Acc: 0.3995 | F1: 0.2559 | Var: 0.0058 | Malicious: 6
Round  4/30 | Acc: 0.1455 | F1: 0.2199 | Var: 0.0034 | Malicious: 6
Round  5/30 | Acc: 0.5818 | F1: 0.2865 | Var: 0.0329 | Malicious: 6
Round  6/30 | Acc: 0.3223 | F1: 0.2330 | Var: 0.0070 | Malicious: 6
Round  8/30 | Acc: 0.3162 | F1: 0.2185 | Var: 0.0030 | Malicious: 6
Round 10/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.1256 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.1278 | F1: 0.2223 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.1284 | F1: 0.2208 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.1250 | F1: 0.2222 | Var: 0.0000 | Malicious: 6
Round 15/30 | Acc: 0.1314 | F1: 0.2235 | Var: 0.0000 | Malicious: 6
Round 16/30 | Acc: 0.1805 | F1: 0.2244 | Var: 0.0005 | Malicious: 6
Round 17/30 | Acc: 0.2426 | F1: 0.2251 | Var: 0.

{'accuracy': 0.2699142156862745,
 'precision': 0.1368653421633554,
 'recall_tpr': 0.9095354523227384,
 'f1_score': 0.23792772625519668,
 'fpr': 0.8217162872154116,
 'roc_auc': 0.5439095825536634,
 'test_loss': 11.639308800347253,
 'tp': 372,
 'tn': 509,
 'fp': 2346,
 'fn': 37,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.0003117941416762782,
 'comm_bytes_round': 215679104,
 'total_comm_bytes': 4744940288,
 'round_time_sec': 188.9020276069641}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING GOSSIP - Attack: gaussian (40%)


I0000 00:00:1753171482.534422    1676 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753171485.816588    1731 service.cc:148] XLA service 0x7991d000d170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753171485.816627    1731 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753171486.034196    1731 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753171487.421591    1731 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8759 | F1: 0.2541 | Var: 0.0590 | Malicious: 6
Round  2/30 | Acc: 0.1670 | F1: 0.2300 | Var: 0.0908 | Malicious: 6
Round  4/30 | Acc: 0.1667 | F1: 0.2299 | Var: 0.0192 | Malicious: 6
Round  5/30 | Acc: 0.2714 | F1: 0.2422 | Var: 0.0596 | Malicious: 6
Round  6/30 | Acc: 0.2739 | F1: 0.2370 | Var: 0.0157 | Malicious: 6
Round  7/30 | Acc: 0.1900 | F1: 0.2309 | Var: 0.0222 | Malicious: 6
Round  8/30 | Acc: 0.4118 | F1: 0.2264 | Var: 0.0130 | Malicious: 6
Round  9/30 | Acc: 0.2972 | F1: 0.2192 | Var: 0.0382 | Malicious: 6
Round 10/30 | Acc: 0.2607 | F1: 0.2034 | Var: 0.0061 | Malicious: 6
Round 11/30 | Acc: 0.2430 | F1: 0.2251 | Var: 0.0318 | Malicious: 6
Round 13/30 | Acc: 0.1477 | F1: 0.1973 | Var: 0.0023 | Malicious: 6
Round 14/30 | Acc: 0.1924 | F1: 0.2079 | Var: 0.0211 | Malicious: 6
Round 15/30 | Acc: 0.1400 | F1: 0.2214 | Var: 0.0120 | Malicious: 6
Round 16/30 | Acc: 0.1373 | F1: 0.2156 | Var: 0.0009 | Malicious: 6
Round 17/30 | Acc: 0.1458 | F1: 0.2151 | Var: 0.

{'accuracy': 0.1253063725490196,
 'precision': 0.1253063725490196,
 'recall_tpr': 1.0,
 'f1_score': 0.22270623468554315,
 'fpr': 1.0,
 'roc_auc': 0.5,
 'test_loss': 13.944702714529845,
 'tp': 409,
 'tn': 0,
 'fp': 2855,
 'fn': 0,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '12;7;10;5;8;4',
 'client_variance': 0.009787187052789522,
 'comm_bytes_round': 390918376,
 'total_comm_bytes': 11107473856,
 'round_time_sec': 184.74015307426453}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv


I0000 00:00:1753262540.497861   20953 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753262543.250945   21002 service.cc:148] XLA service 0x7ae290009e70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753262543.250998   21002 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753262543.476242   21002 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753262544.919691   21002 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.2867 | F1: 0.2825 | Var: 0.0613 | Malicious: 6
Round  2/30 | Acc: 0.8819 | F1: 0.5849 | Var: 0.0586 | Malicious: 6
Round  3/30 | Acc: 0.9236 | F1: 0.7513 | Var: 0.0176 | Malicious: 6
Round  4/30 | Acc: 0.9127 | F1: 0.7550 | Var: 0.0224 | Malicious: 6
Round  5/30 | Acc: 0.6658 | F1: 0.2950 | Var: 0.0437 | Malicious: 6
Round  6/30 | Acc: 0.8409 | F1: 0.5615 | Var: 0.0499 | Malicious: 6
Round  7/30 | Acc: 0.9208 | F1: 0.7191 | Var: 0.0544 | Malicious: 6
Round  8/30 | Acc: 0.9164 | F1: 0.7115 | Var: 0.0724 | Malicious: 6
Round  9/30 | Acc: 0.9355 | F1: 0.7667 | Var: 0.0136 | Malicious: 6
Round 10/30 | Acc: 0.9233 | F1: 0.7921 | Var: 0.0690 | Malicious: 6
Round 11/30 | Acc: 0.9307 | F1: 0.7438 | Var: 0.0589 | Malicious: 6
Round 12/30 | Acc: 0.4476 | F1: 0.1892 | Var: 0.0237 | Malicious: 6
Round 13/30 | Acc: 0.8620 | F1: 0.4265 | Var: 0.0447 | Malicious: 6
Round 14/30 | Acc: 0.9065 | F1: 0.7386 | Var: 0.0512 | Malicious: 6
Round 15/30 | Acc: 0.9014 | F1: 0.7416 | Var: 0.

{'accuracy': 0.9183161394106131,
 'precision': 0.9354174095645628,
 'recall_tpr': 0.5545989499524308,
 'f1_score': 0.6963433787799465,
 'fpr': 0.0077802067721210845,
 'roc_auc': 0.7799373944863054,
 'test_loss': 1.2974563668671628,
 'tp': 47218,
 'tn': 415752,
 'fp': 3260,
 'fn': 37921,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.007725264709068462,
 'comm_bytes_round': 2195584,
 'total_comm_bytes': 64769728,
 'round_time_sec': 226.20868396759033}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING FCWTL - Attack: gaussian (40%)


I0000 00:00:1753276999.442597   60626 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753277002.895524   60679 service.cc:148] XLA service 0x7ad8d400c4d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753277002.896063   60679 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753277003.218904   60679 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753277004.803726   60679 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8231 | F1: 0.5974 | Var: 0.0000 | Malicious: 6
Round  2/30 | Acc: 0.6884 | F1: 0.5172 | Var: 0.0000 | Malicious: 6
Round  3/30 | Acc: 0.9086 | F1: 0.7254 | Var: 0.0000 | Malicious: 6
Round  4/30 | Acc: 0.8315 | F1: 0.6198 | Var: 0.0000 | Malicious: 6
Round  5/30 | Acc: 0.8892 | F1: 0.7074 | Var: 0.0000 | Malicious: 6
Round  6/30 | Acc: 0.8467 | F1: 0.6547 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.9334 | F1: 0.7961 | Var: 0.0000 | Malicious: 6
Round 15/30 | Acc: 0.9364 | F1: 0.8251 | Var: 0.0000 | Malicious: 6
Round 16/30 | Acc: 0.9366 | F1: 0.8011 | Var: 0.0000 | Malicious: 6
Round 17/30 | Acc: 0.9459 | F1: 0.8440 | Var: 0.0000 | Malicious: 6
Round 20/30 | Acc: 0.9001 | F1: 0.7161 | Var: 0.0000 | Malicious: 6
Round 21/30 | Acc: 0.9161 | F1: 0.7173 | Var: 0.0000 | Malicious: 6
Round 22/30 | Acc: 0.9283 | F1: 0.8020 | Var: 0.0000 | Malicious: 6
Round 23/30 | Acc: 0.9154 | F1: 0.7725 | Var: 0.0000 | Malicious: 6
Round 24/30 | Acc: 0.9244 | F1: 0.7852 | Var: 0.

{'accuracy': 0.9163405408300291,
 'precision': 0.7026318271861145,
 'recall_tpr': 0.8748752040780371,
 'f1_score': 0.7793501404663379,
 'fpr': 0.07523412217311198,
 'roc_auc': 0.8998205409524626,
 'test_loss': 1.3337314290055209,
 'tp': 74486,
 'tn': 387488,
 'fp': 31524,
 'fn': 10653,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '10;12;9;4;14;3',
 'client_variance': 0.0,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 41716096,
 'round_time_sec': 153.71254992485046}

In [3]:
print("hassssssssssssss")

hassssssssssssss


In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING RING - Attack: gaussian (40%)


I0000 00:00:1753296165.560400      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753296168.639763      95 service.cc:148] XLA service 0x7a0d3c00b4d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753296168.640227      95 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753296168.928037      95 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753296170.393869      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8231 | F1: 0.5974 | Var: 0.0553 | Malicious: 6
Round  2/30 | Acc: 0.4347 | F1: 0.3395 | Var: 0.0203 | Malicious: 6
Round  3/30 | Acc: 0.9175 | F1: 0.7294 | Var: 0.0133 | Malicious: 6
Round  4/30 | Acc: 0.8284 | F1: 0.6254 | Var: 0.0009 | Malicious: 6
Round  5/30 | Acc: 0.8844 | F1: 0.7177 | Var: 0.0007 | Malicious: 6
Round  6/30 | Acc: 0.8524 | F1: 0.6646 | Var: 0.0012 | Malicious: 6
Round  7/30 | Acc: 0.8708 | F1: 0.7019 | Var: 0.0008 | Malicious: 6
Round  8/30 | Acc: 0.9176 | F1: 0.7702 | Var: 0.0007 | Malicious: 6
Round  9/30 | Acc: 0.9480 | F1: 0.8414 | Var: 0.0002 | Malicious: 6
Round 10/30 | Acc: 0.9053 | F1: 0.7371 | Var: 0.0001 | Malicious: 6
Round 11/30 | Acc: 0.8842 | F1: 0.7162 | Var: 0.0003 | Malicious: 6
Round 12/30 | Acc: 0.9144 | F1: 0.7302 | Var: 0.0001 | Malicious: 6
Round 13/30 | Acc: 0.8344 | F1: 0.6585 | Var: 0.0010 | Malicious: 6
Round 14/30 | Acc: 0.9318 | F1: 0.7998 | Var: 0.0001 | Malicious: 6
Round 15/30 | Acc: 0.9378 | F1: 0.8257 | Var: 0.

{'accuracy': 0.9131807732207216,
 'precision': 0.7003981902206032,
 'recall_tpr': 0.849117325784893,
 'f1_score': 0.7676208881055026,
 'fpr': 0.07380218227640259,
 'roc_auc': 0.8876575717542452,
 'test_loss': 1.3841056608500188,
 'tp': 72293,
 'tn': 388088,
 'fp': 30924,
 'fn': 12846,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 4.9042766939768186e-05,
 'comm_bytes_round': 2195584,
 'total_comm_bytes': 48302848,
 'round_time_sec': 221.26619243621826}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING GOSSIP - Attack: gaussian (40%)


I0000 00:00:1753301656.624477   21815 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753301659.236262   21871 service.cc:148] XLA service 0x7988d800b4c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753301659.236305   21871 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753301659.444394   21871 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753301660.794113   21871 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.7822 | F1: 0.2429 | Var: 0.0681 | Malicious: 6
Round  2/30 | Acc: 0.9040 | F1: 0.6673 | Var: 0.0696 | Malicious: 6
Round  3/30 | Acc: 0.9258 | F1: 0.7457 | Var: 0.0063 | Malicious: 6
Round  4/30 | Acc: 0.8420 | F1: 0.6742 | Var: 0.0423 | Malicious: 6
Round  5/30 | Acc: 0.8363 | F1: 0.0596 | Var: 0.0770 | Malicious: 6
Round  6/30 | Acc: 0.8542 | F1: 0.2433 | Var: 0.0561 | Malicious: 6
Round  7/30 | Acc: 0.8359 | F1: 0.1922 | Var: 0.0466 | Malicious: 6
Round  8/30 | Acc: 0.9166 | F1: 0.6800 | Var: 0.0504 | Malicious: 6
Round  9/30 | Acc: 0.9197 | F1: 0.6887 | Var: 0.0061 | Malicious: 6
Round 10/30 | Acc: 0.7753 | F1: 0.5630 | Var: 0.0117 | Malicious: 6
Round 11/30 | Acc: 0.8383 | F1: 0.6430 | Var: 0.0378 | Malicious: 6
Round 12/30 | Acc: 0.8895 | F1: 0.5209 | Var: 0.0360 | Malicious: 6
Round 13/30 | Acc: 0.8790 | F1: 0.6490 | Var: 0.0154 | Malicious: 6
Round 14/30 | Acc: 0.9280 | F1: 0.7659 | Var: 0.0780 | Malicious: 6
Round 15/30 | Acc: 0.8567 | F1: 0.6638 | Var: 0.

{'accuracy': 0.8898762473941338,
 'precision': 0.7127812419183357,
 'recall_tpr': 0.5827059279531119,
 'f1_score': 0.6412133823614944,
 'fpr': 0.04770985079186276,
 'roc_auc': 0.8350975763326182,
 'test_loss': 1.7554860089825561,
 'tp': 49611,
 'tn': 399021,
 'fp': 19991,
 'fn': 35528,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '12;7;10;5;8;4',
 'client_variance': 0.0016343735648269922,
 'comm_bytes_round': 3979496,
 'total_comm_bytes': 113072576,
 'round_time_sec': 219.18030786514282}

In [5]:
1

1

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING FEDAVG - Attack: gaussian (40%)


I0000 00:00:1753030674.553557   21988 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753030677.723229   22042 service.cc:148] XLA service 0x7d2bd400ae30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753030677.723293   22042 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753030677.939002   22042 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753030679.335604   22042 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8597 | F1: 0.0172 | Var: 0.0981 | Malicious: 6
Round  2/30 | Acc: 0.7497 | F1: 0.0577 | Var: 0.0805 | Malicious: 6
Round  3/30 | Acc: 0.1829 | F1: 0.1438 | Var: 0.0464 | Malicious: 6
Round  8/30 | Acc: 0.3928 | F1: 0.1261 | Var: 0.0327 | Malicious: 6
Round  9/30 | Acc: 0.3569 | F1: 0.1294 | Var: 0.0095 | Malicious: 6
Round 10/30 | Acc: 0.4418 | F1: 0.0989 | Var: 0.0102 | Malicious: 6
Round 11/30 | Acc: 0.2341 | F1: 0.1427 | Var: 0.0251 | Malicious: 6
Round 12/30 | Acc: 0.3511 | F1: 0.1313 | Var: 0.0310 | Malicious: 6
Round 13/30 | Acc: 0.3977 | F1: 0.1247 | Var: 0.0302 | Malicious: 6
Round 14/30 | Acc: 0.4317 | F1: 0.1595 | Var: 0.0331 | Malicious: 6
Round 15/30 | Acc: 0.5490 | F1: 0.1111 | Var: 0.0339 | Malicious: 6
Round 16/30 | Acc: 0.7142 | F1: 0.1417 | Var: 0.0328 | Malicious: 6
Round 17/30 | Acc: 0.7849 | F1: 0.1069 | Var: 0.0236 | Malicious: 6
Round 18/30 | Acc: 0.8508 | F1: 0.0507 | Var: 0.0252 | Malicious: 6
Round 19/30 | Acc: 0.6645 | F1: 0.1105 | Var: 0.

{'accuracy': 0.8115808823529411,
 'precision': 0.17197452229299362,
 'recall_tpr': 0.13202933985330073,
 'f1_score': 0.14937759336099582,
 'fpr': 0.09106830122591944,
 'roc_auc': 0.5204805193136905,
 'test_loss': 3.00385024044297,
 'tp': 54,
 'tn': 2595,
 'fp': 260,
 'fn': 355,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'gaussian',
 'attack_pct': 0.4,
 'sigma': 5,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.039930290232923236,
 'comm_bytes_round': 215679104,
 'total_comm_bytes': 6362533568,
 'round_time_sec': 186.0801055431366}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING FCWTL - Attack: gaussian (20%)


I0000 00:00:1753085229.163403      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753085233.048315     100 service.cc:148] XLA service 0x78bc8008db90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753085233.049236     100 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753085233.357136     100 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753085234.909926     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.7702 | F1: 0.3932 | Var: 0.0000 | Malicious: 3
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  3/30 | Acc: 0.7999 | F1: 0.1091 | Var: 0.0000 | Malicious: 3
Round  4/30 | Acc: 0.6458 | F1: 0.3200 | Var: 0.0000 | Malicious: 3
Round  5/30 | Acc: 0.3395 | F1: 0.2545 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.1535 | F1: 0.2184 | Var: 0.0000 | Malicious: 3
Round  9/30 | Acc: 0.5450 | F1: 0.1441 | Var: 0.0000 | Malicious: 3
Round 10/30 | Acc: 0.6719 | F1: 0.2931 | Var: 0.0000 | Malicious: 3
Round 11/30 | Acc: 0.5812 | F1: 0.1375 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.3729 | F1: 0.2142 | Var: 0.0000 | Malicious: 3
Round 13/30 | Acc: 0.6222 | F1: 0.1136 | Var: 0.0000 | Malicious: 3
Round 14/30 | Acc: 0.5695 | F1: 0.2085 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.4684 | F1: 0.1734 | Var: 0.0000 | Malicious: 3
Round 16/30 | Acc: 0.4384 | F1: 0.2203 | Var: 0.0000 | Malicious: 3
Round 17/30 | Acc: 0.2525 | F1: 0.2341 | Var: 0.

{'accuracy': 0.15349264705882354,
 'precision': 0.12611181702668361,
 'recall_tpr': 0.9706601466992665,
 'f1_score': 0.22322181613719427,
 'fpr': 0.9635726795096322,
 'roc_auc': 0.5035437335948172,
 'test_loss': 13.495346273629849,
 'tp': 397,
 'tn': 104,
 'fp': 2751,
 'fn': 12,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '12;5;8',
 'client_variance': 0.0,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3990063424,
 'round_time_sec': 119.70078492164612}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING RING - Attack: gaussian (20%)


I0000 00:00:1753088219.532372   10875 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753088222.748550   10925 service.cc:148] XLA service 0x7dfbfc00a6d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753088222.748597   10925 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753088222.960699   10925 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753088224.342922   10925 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.7702 | F1: 0.3932 | Var: 0.0032 | Malicious: 3
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0027 | Malicious: 3
Round  4/30 | Acc: 0.6320 | F1: 0.3226 | Var: 0.0022 | Malicious: 3
Round  5/30 | Acc: 0.3165 | F1: 0.2506 | Var: 0.0096 | Malicious: 3
Round  6/30 | Acc: 0.4213 | F1: 0.2601 | Var: 0.0005 | Malicious: 3
Round  7/30 | Acc: 0.1786 | F1: 0.2181 | Var: 0.0087 | Malicious: 3
Round  8/30 | Acc: 0.1544 | F1: 0.2190 | Var: 0.0095 | Malicious: 3
Round  9/30 | Acc: 0.5383 | F1: 0.1442 | Var: 0.0371 | Malicious: 3
Round 10/30 | Acc: 0.6703 | F1: 0.2967 | Var: 0.0367 | Malicious: 3
Round 11/30 | Acc: 0.5797 | F1: 0.1404 | Var: 0.0039 | Malicious: 3
Round 12/30 | Acc: 0.3732 | F1: 0.2161 | Var: 0.0046 | Malicious: 3
Round 13/30 | Acc: 0.6167 | F1: 0.1134 | Var: 0.0071 | Malicious: 3
Round 14/30 | Acc: 0.5674 | F1: 0.2085 | Var: 0.0043 | Malicious: 3
Round 15/30 | Acc: 0.4663 | F1: 0.1728 | Var: 0.0048 | Malicious: 3
Round 16/30 | Acc: 0.4418 | F1: 0.2227 | Var: 0.

{'accuracy': 0.15410539215686275,
 'precision': 0.12619198982835347,
 'recall_tpr': 0.9706601466992665,
 'f1_score': 0.22334739803094233,
 'fpr': 0.9628721541155867,
 'roc_auc': 0.5038939962918398,
 'test_loss': 13.485577655349413,
 'tp': 397,
 'tn': 106,
 'fp': 2749,
 'fn': 12,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '2;6;3',
 'client_variance': 0.00012538830551283705,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3882223872,
 'round_time_sec': 187.49802327156067}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING GOSSIP - Attack: gaussian (20%)


I0000 00:00:1753093108.223153   31813 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753093111.474227   31867 service.cc:148] XLA service 0x7d785c08c240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753093111.474289   31867 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753093111.687849   31867 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753093113.090966   31867 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.5242 | F1: 0.2892 | Var: 0.0763 | Malicious: 3
Round  2/30 | Acc: 0.8416 | F1: 0.4447 | Var: 0.0166 | Malicious: 3
Round  3/30 | Acc: 0.4513 | F1: 0.2717 | Var: 0.0767 | Malicious: 3
Round  4/30 | Acc: 0.1296 | F1: 0.2236 | Var: 0.0181 | Malicious: 3
Round  5/30 | Acc: 0.1299 | F1: 0.2223 | Var: 0.0030 | Malicious: 3
Round  6/30 | Acc: 0.2721 | F1: 0.2443 | Var: 0.0063 | Malicious: 3
Round  7/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0076 | Malicious: 3
Round  8/30 | Acc: 0.1256 | F1: 0.2228 | Var: 0.0008 | Malicious: 3
Round  9/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0156 | Malicious: 3
Round 10/30 | Acc: 0.1268 | F1: 0.2230 | Var: 0.0003 | Malicious: 3
Round 11/30 | Acc: 0.1284 | F1: 0.2233 | Var: 0.0009 | Malicious: 3
Round 12/30 | Acc: 0.1268 | F1: 0.2230 | Var: 0.0003 | Malicious: 3
Round 13/30 | Acc: 0.1278 | F1: 0.2232 | Var: 0.0044 | Malicious: 3
Round 14/30 | Acc: 0.1275 | F1: 0.2227 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.1278 | F1: 0.2206 | Var: 0.

{'accuracy': 0.19791666666666666,
 'precision': 0.13097226862679587,
 'recall_tpr': 0.9584352078239609,
 'f1_score': 0.23045267489711938,
 'fpr': 0.9110332749562171,
 'roc_auc': 0.5238469805899657,
 'test_loss': 12.785339245124323,
 'tp': 392,
 'tn': 254,
 'fp': 2601,
 'fn': 17,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '9;8;5',
 'client_variance': 0.0056456300861848,
 'comm_bytes_round': 350478544,
 'total_comm_bytes': 10177357720,
 'round_time_sec': 185.75827312469482}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv


I0000 00:00:1753191660.829198   20955 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753191663.411887   21005 service.cc:148] XLA service 0x7cedc000c200 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753191663.411937   21005 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753191663.622838   21005 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753191665.016375   21005 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.9562 | F1: 0.8514 | Var: 0.0156 | Malicious: 0
Round  2/30 | Acc: 0.9583 | F1: 0.8598 | Var: 0.0002 | Malicious: 0
Round  4/30 | Acc: 0.9627 | F1: 0.8767 | Var: 0.0002 | Malicious: 0
Round  5/30 | Acc: 0.9678 | F1: 0.8957 | Var: 0.0002 | Malicious: 0
Round  6/30 | Acc: 0.9677 | F1: 0.8952 | Var: 0.0001 | Malicious: 0
Round  7/30 | Acc: 0.9692 | F1: 0.9002 | Var: 0.0002 | Malicious: 0
Round  8/30 | Acc: 0.9752 | F1: 0.9213 | Var: 0.0002 | Malicious: 0
Round  9/30 | Acc: 0.9717 | F1: 0.9089 | Var: 0.0002 | Malicious: 0
Round 10/30 | Acc: 0.9744 | F1: 0.9189 | Var: 0.0002 | Malicious: 0
Round 11/30 | Acc: 0.9736 | F1: 0.9158 | Var: 0.0002 | Malicious: 0
Round 12/30 | Acc: 0.9718 | F1: 0.9094 | Var: 0.0001 | Malicious: 0
Round 13/30 | Acc: 0.9747 | F1: 0.9197 | Var: 0.0001 | Malicious: 0
Round 14/30 | Acc: 0.9752 | F1: 0.9214 | Var: 0.0001 | Malicious: 0
Round 15/30 | Acc: 0.9776 | F1: 0.9297 | Var: 0.0001 | Malicious: 0
Round 16/30 | Acc: 0.9758 | F1: 0.9235 | Var: 0.

{'accuracy': 0.9748507887517827,
 'precision': 0.9944590020744623,
 'recall_tpr': 0.8558474964469867,
 'f1_score': 0.9199613663192582,
 'fpr': 0.0009689459967733621,
 'roc_auc': 0.9977069849975239,
 'test_loss': 0.0713633738655416,
 'tp': 72866,
 'tn': 418606,
 'fp': 406,
 'fn': 12273,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 0.00015381004141809884,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 32933760,
 'round_time_sec': 218.17321729660034}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2024 from /kaggle/input/ciciomt2024/wustl-ehms-2020_with_attacks_categories.csv
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING RING - Attack: none (0%)


I0000 00:00:1753186666.224379      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753186669.936692      97 service.cc:148] XLA service 0x7c64ac00b630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753186669.937330      97 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753186670.221052      97 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753186671.711053      97 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.8928 | F1: 0.2585 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.9357 | F1: 0.6591 | Var: 0.0004 | Malicious: 0
Round  7/30 | Acc: 0.9372 | F1: 0.6699 | Var: 0.0003 | Malicious: 0
Round  8/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round  9/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 10/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 11/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 12/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 13/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 14/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 15/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.

{'accuracy': 0.9356617647058824,
 'precision': 0.9128630705394191,
 'recall_tpr': 0.5378973105134475,
 'f1_score': 0.676923076923077,
 'fpr': 0.007355516637478109,
 'roc_auc': 0.8569116935501137,
 'test_loss': 0.2712423780737428,
 'tp': 220,
 'tn': 2834,
 'fp': 21,
 'fn': 189,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 0.00014769793402110278,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 184.5548448562622}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING RING - Attack: none (0%)


I0000 00:00:1753175704.564178   21978 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753175707.739567   22032 service.cc:148] XLA service 0x781b6800ac90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753175707.739618   22032 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753175707.954311   22032 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753175709.328292   22032 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.8928 | F1: 0.2585 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.9357 | F1: 0.6591 | Var: 0.0004 | Malicious: 0
Round  7/30 | Acc: 0.9372 | F1: 0.6699 | Var: 0.0003 | Malicious: 0
Round  8/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round  9/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 10/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 11/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 12/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 13/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 14/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 15/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 16/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.

{'accuracy': 0.9356617647058824,
 'precision': 0.9128630705394191,
 'recall_tpr': 0.5378973105134475,
 'f1_score': 0.676923076923077,
 'fpr': 0.007355516637478109,
 'roc_auc': 0.8569116935501137,
 'test_loss': 0.2712423780737428,
 'tp': 220,
 'tn': 2834,
 'fp': 21,
 'fn': 189,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 0.00014769793402110278,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 186.29116487503052}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FEDAVG - Attack: gaussian (20%)


I0000 00:00:1753098894.962930   52725 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753098898.196214   52777 service.cc:148] XLA service 0x79da9008d4e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753098898.196260   52777 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753098898.406683   52777 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753098900.043298   52777 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.5432 | F1: 0.2787 | Var: 0.1058 | Malicious: 3
Round  2/30 | Acc: 0.7760 | F1: 0.2313 | Var: 0.0108 | Malicious: 3
Round  3/30 | Acc: 0.7353 | F1: 0.1970 | Var: 0.0560 | Malicious: 3
Round  4/30 | Acc: 0.7344 | F1: 0.2224 | Var: 0.0312 | Malicious: 3
Round  5/30 | Acc: 0.7537 | F1: 0.2328 | Var: 0.0235 | Malicious: 3
Round  6/30 | Acc: 0.5631 | F1: 0.1729 | Var: 0.0066 | Malicious: 3
Round  7/30 | Acc: 0.6115 | F1: 0.2114 | Var: 0.0270 | Malicious: 3
Round  8/30 | Acc: 0.7013 | F1: 0.2181 | Var: 0.0288 | Malicious: 3
Round  9/30 | Acc: 0.8263 | F1: 0.1818 | Var: 0.0231 | Malicious: 3
Round 10/30 | Acc: 0.8615 | F1: 0.0776 | Var: 0.0565 | Malicious: 3
Round 11/30 | Acc: 0.7619 | F1: 0.3809 | Var: 0.0480 | Malicious: 3
Round 12/30 | Acc: 0.7184 | F1: 0.0981 | Var: 0.0326 | Malicious: 3
Round 13/30 | Acc: 0.3729 | F1: 0.1446 | Var: 0.0302 | Malicious: 3
Round 14/30 | Acc: 0.3067 | F1: 0.2279 | Var: 0.0042 | Malicious: 3
Round 15/30 | Acc: 0.2684 | F1: 0.2077 | Var: 0.

{'accuracy': 0.7610294117647058,
 'precision': 0.12974051896207583,
 'recall_tpr': 0.15892420537897312,
 'f1_score': 0.14285714285714285,
 'fpr': 0.15271453590192643,
 'roc_auc': 0.5031048347385234,
 'test_loss': 3.8097612485788335,
 'tp': 65,
 'tn': 2419,
 'fp': 436,
 'fn': 344,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '2;6;3',
 'client_variance': 0.021263800114272292,
 'comm_bytes_round': 215679104,
 'total_comm_bytes': 6362533568,
 'round_time_sec': 195.25966215133667}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FCWTL - Attack: gaussian (20%)


I0000 00:00:1753103540.659480   73635 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753103544.072688   73689 service.cc:148] XLA service 0x7a14bc00a7b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753103544.072731   73689 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753103544.301262   73689 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753103545.816136   73689 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.7702 | F1: 0.3932 | Var: 0.0000 | Malicious: 3
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  3/30 | Acc: 0.7999 | F1: 0.1091 | Var: 0.0000 | Malicious: 3
Round  4/30 | Acc: 0.6458 | F1: 0.3200 | Var: 0.0000 | Malicious: 3
Round  5/30 | Acc: 0.3395 | F1: 0.2545 | Var: 0.0000 | Malicious: 3
Round  6/30 | Acc: 0.4249 | F1: 0.2596 | Var: 0.0000 | Malicious: 3
Round  7/30 | Acc: 0.1808 | F1: 0.2177 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.1535 | F1: 0.2184 | Var: 0.0000 | Malicious: 3
Round  9/30 | Acc: 0.5450 | F1: 0.1441 | Var: 0.0000 | Malicious: 3
Round 10/30 | Acc: 0.6719 | F1: 0.2931 | Var: 0.0000 | Malicious: 3
Round 11/30 | Acc: 0.5812 | F1: 0.1375 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.3729 | F1: 0.2142 | Var: 0.0000 | Malicious: 3
Round 13/30 | Acc: 0.6222 | F1: 0.1136 | Var: 0.0000 | Malicious: 3
Round 14/30 | Acc: 0.5695 | F1: 0.2085 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.4684 | F1: 0.1734 | Var: 0.

{'accuracy': 0.15349264705882354,
 'precision': 0.12611181702668361,
 'recall_tpr': 0.9706601466992665,
 'f1_score': 0.22322181613719427,
 'fpr': 0.9635726795096322,
 'roc_auc': 0.5035437335948172,
 'test_loss': 13.495346273629849,
 'tp': 397,
 'tn': 104,
 'fp': 2751,
 'fn': 12,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '12;5;8',
 'client_variance': 0.0,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3990063424,
 'round_time_sec': 121.80449748039246}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING RING - Attack: gaussian (20%)


I0000 00:00:1753106736.868924   84466 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753106740.079697   84520 service.cc:148] XLA service 0x79971000c210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753106740.079741   84520 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753106740.318671   84520 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753106741.725467   84520 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.7702 | F1: 0.3932 | Var: 0.0032 | Malicious: 3
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0027 | Malicious: 3
Round  3/30 | Acc: 0.7907 | F1: 0.1072 | Var: 0.0015 | Malicious: 3
Round  4/30 | Acc: 0.6320 | F1: 0.3226 | Var: 0.0022 | Malicious: 3
Round  6/30 | Acc: 0.4213 | F1: 0.2601 | Var: 0.0005 | Malicious: 3
Round  7/30 | Acc: 0.1786 | F1: 0.2181 | Var: 0.0087 | Malicious: 3
Round  8/30 | Acc: 0.1544 | F1: 0.2190 | Var: 0.0095 | Malicious: 3
Round  9/30 | Acc: 0.5383 | F1: 0.1442 | Var: 0.0371 | Malicious: 3
Round 10/30 | Acc: 0.6703 | F1: 0.2967 | Var: 0.0367 | Malicious: 3
Round 11/30 | Acc: 0.5797 | F1: 0.1404 | Var: 0.0039 | Malicious: 3
Round 12/30 | Acc: 0.3732 | F1: 0.2161 | Var: 0.0046 | Malicious: 3
Round 13/30 | Acc: 0.6167 | F1: 0.1134 | Var: 0.0071 | Malicious: 3
Round 14/30 | Acc: 0.5674 | F1: 0.2085 | Var: 0.0043 | Malicious: 3
Round 15/30 | Acc: 0.4663 | F1: 0.1728 | Var: 0.0048 | Malicious: 3
Round 16/30 | Acc: 0.4418 | F1: 0.2227 | Var: 0.

{'accuracy': 0.15410539215686275,
 'precision': 0.12619198982835347,
 'recall_tpr': 0.9706601466992665,
 'f1_score': 0.22334739803094233,
 'fpr': 0.9628721541155867,
 'roc_auc': 0.5038939962918398,
 'test_loss': 13.485577655349413,
 'tp': 397,
 'tn': 106,
 'fp': 2749,
 'fn': 12,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '2;6;3',
 'client_variance': 0.00012538830551283705,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3882223872,
 'round_time_sec': 188.25460767745972}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING GOSSIP - Attack: gaussian (20%)


I0000 00:00:1753171254.277182    1041 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753171257.463954    1095 service.cc:148] XLA service 0x7e3904009d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753171257.463999    1095 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753171257.674555    1095 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753171259.047199    1095 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.5242 | F1: 0.2892 | Var: 0.0763 | Malicious: 3
Round  2/30 | Acc: 0.8416 | F1: 0.4447 | Var: 0.0166 | Malicious: 3
Round  3/30 | Acc: 0.4513 | F1: 0.2717 | Var: 0.0767 | Malicious: 3
Round  4/30 | Acc: 0.1296 | F1: 0.2236 | Var: 0.0181 | Malicious: 3
Round  5/30 | Acc: 0.1299 | F1: 0.2223 | Var: 0.0030 | Malicious: 3
Round  6/30 | Acc: 0.2721 | F1: 0.2443 | Var: 0.0063 | Malicious: 3
Round  7/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0076 | Malicious: 3
Round  8/30 | Acc: 0.1256 | F1: 0.2228 | Var: 0.0008 | Malicious: 3
Round  9/30 | Acc: 0.1265 | F1: 0.2229 | Var: 0.0156 | Malicious: 3
Round 10/30 | Acc: 0.1268 | F1: 0.2230 | Var: 0.0003 | Malicious: 3
Round 11/30 | Acc: 0.1284 | F1: 0.2233 | Var: 0.0009 | Malicious: 3
Round 12/30 | Acc: 0.1268 | F1: 0.2230 | Var: 0.0003 | Malicious: 3
Round 13/30 | Acc: 0.1278 | F1: 0.2232 | Var: 0.0044 | Malicious: 3
Round 14/30 | Acc: 0.1275 | F1: 0.2227 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.1278 | F1: 0.2206 | Var: 0.

{'accuracy': 0.19791666666666666,
 'precision': 0.13097226862679587,
 'recall_tpr': 0.9584352078239609,
 'f1_score': 0.23045267489711938,
 'fpr': 0.9110332749562171,
 'roc_auc': 0.5238469805899657,
 'test_loss': 12.785339245124323,
 'tp': 392,
 'tn': 254,
 'fp': 2601,
 'fn': 17,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '9;8;5',
 'client_variance': 0.0056456300861848,
 'comm_bytes_round': 350478544,
 'total_comm_bytes': 10177357720,
 'round_time_sec': 185.95133256912231}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING FEDAVG - Attack: gaussian (20%)


I0000 00:00:1753258018.716446      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753258021.945894      98 service.cc:148] XLA service 0x7e6c6800bf80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753258021.946560      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753258022.244127      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753258023.757475      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.1765 | F1: 0.2861 | Var: 0.0577 | Malicious: 3
Round  2/30 | Acc: 0.7280 | F1: 0.2298 | Var: 0.0284 | Malicious: 3
Round  3/30 | Acc: 0.5591 | F1: 0.2669 | Var: 0.0386 | Malicious: 3
Round  4/30 | Acc: 0.5915 | F1: 0.0774 | Var: 0.0416 | Malicious: 3
Round  5/30 | Acc: 0.8817 | F1: 0.6967 | Var: 0.0188 | Malicious: 3
Round  6/30 | Acc: 0.9312 | F1: 0.7513 | Var: 0.0067 | Malicious: 3
Round  7/30 | Acc: 0.9451 | F1: 0.8122 | Var: 0.0084 | Malicious: 3
Round  8/30 | Acc: 0.8256 | F1: 0.1050 | Var: 0.0403 | Malicious: 3
Round  9/30 | Acc: 0.9424 | F1: 0.8091 | Var: 0.0422 | Malicious: 3
Round 10/30 | Acc: 0.9261 | F1: 0.7632 | Var: 0.0134 | Malicious: 3
Round 11/30 | Acc: 0.9173 | F1: 0.6777 | Var: 0.0073 | Malicious: 3
Round 12/30 | Acc: 0.9341 | F1: 0.7817 | Var: 0.0358 | Malicious: 3
Round 13/30 | Acc: 0.8233 | F1: 0.0147 | Var: 0.0379 | Malicious: 3
Round 14/30 | Acc: 0.9514 | F1: 0.8462 | Var: 0.0351 | Malicious: 3
Round 15/30 | Acc: 0.9258 | F1: 0.7195 | Var: 0.

{'accuracy': 0.9193158398971737,
 'precision': 0.9835138544521292,
 'recall_tpr': 0.5311314438741352,
 'f1_score': 0.6897656291708931,
 'fpr': 0.001809017402842878,
 'roc_auc': 0.8220166085255782,
 'test_loss': 0.9476226860738924,
 'tp': 45220,
 'tn': 418254,
 'fp': 758,
 'fn': 39919,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '2;6;3',
 'client_variance': 0.00935372274497486,
 'comm_bytes_round': 2195584,
 'total_comm_bytes': 64769728,
 'round_time_sec': 227.09004139900208}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING FCWTL - Attack: gaussian (20%)


I0000 00:00:1753263709.332834   21865 service.cc:148] XLA service 0x7ab548009ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753263709.332874   21865 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753263709.569337   21865 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753263711.036772   21865 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8126 | F1: 0.6078 | Var: 0.0000 | Malicious: 3
Round  3/30 | Acc: 0.7599 | F1: 0.5775 | Var: 0.0000 | Malicious: 3
Round  4/30 | Acc: 0.8674 | F1: 0.7135 | Var: 0.0000 | Malicious: 3
Round  5/30 | Acc: 0.5450 | F1: 0.4207 | Var: 0.0000 | Malicious: 3
Round  6/30 | Acc: 0.8795 | F1: 0.7233 | Var: 0.0000 | Malicious: 3
Round  7/30 | Acc: 0.8622 | F1: 0.7038 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.8259 | F1: 0.6266 | Var: 0.0000 | Malicious: 3
Round  9/30 | Acc: 0.8935 | F1: 0.7184 | Var: 0.0000 | Malicious: 3
Round 10/30 | Acc: 0.9153 | F1: 0.7831 | Var: 0.0000 | Malicious: 3
Round 11/30 | Acc: 0.9311 | F1: 0.7776 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.8863 | F1: 0.7389 | Var: 0.0000 | Malicious: 3
Round 13/30 | Acc: 0.9048 | F1: 0.7635 | Var: 0.0000 | Malicious: 3
Round 14/30 | Acc: 0.8743 | F1: 0.7024 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.9301 | F1: 0.8014 | Var: 0.0000 | Malicious: 3
Round 16/30 | Acc: 0.9433 | F1: 0.8251 | Var: 0.

{'accuracy': 0.9143550245858879,
 'precision': 0.67866236343044,
 'recall_tpr': 0.9360692514593782,
 'f1_score': 0.7868489904724294,
 'fpr': 0.09005708667054882,
 'roc_auc': 0.9230060061069235,
 'test_loss': 1.365385293460224,
 'tp': 79696,
 'tn': 381277,
 'fp': 37735,
 'fn': 5443,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '12;5;8',
 'client_variance': 0.0,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 40618304,
 'round_time_sec': 145.41478157043457}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING RING - Attack: gaussian (20%)


I0000 00:00:1753278232.803491      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753278235.890936      98 service.cc:148] XLA service 0x7b5bfc00b170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753278235.891520      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753278236.177413      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753278237.637768      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8126 | F1: 0.6078 | Var: 0.0683 | Malicious: 3
Round  2/30 | Acc: 0.8128 | F1: 0.6315 | Var: 0.0328 | Malicious: 3
Round  3/30 | Acc: 0.7405 | F1: 0.5572 | Var: 0.0098 | Malicious: 3
Round  4/30 | Acc: 0.4947 | F1: 0.3953 | Var: 0.0143 | Malicious: 3
Round  5/30 | Acc: 0.8182 | F1: 0.6092 | Var: 0.0201 | Malicious: 3
Round  7/30 | Acc: 0.8618 | F1: 0.6862 | Var: 0.0015 | Malicious: 3
Round  8/30 | Acc: 0.8490 | F1: 0.6610 | Var: 0.0010 | Malicious: 3
Round  9/30 | Acc: 0.8844 | F1: 0.7220 | Var: 0.0009 | Malicious: 3
Round 10/30 | Acc: 0.9121 | F1: 0.7768 | Var: 0.0004 | Malicious: 3
Round 12/30 | Acc: 0.8858 | F1: 0.7276 | Var: 0.0005 | Malicious: 3
Round 13/30 | Acc: 0.9093 | F1: 0.7681 | Var: 0.0004 | Malicious: 3
Round 14/30 | Acc: 0.8917 | F1: 0.7281 | Var: 0.0003 | Malicious: 3
Round 15/30 | Acc: 0.9027 | F1: 0.7483 | Var: 0.0003 | Malicious: 3
Round 16/30 | Acc: 0.9020 | F1: 0.7531 | Var: 0.0002 | Malicious: 3
Round 17/30 | Acc: 0.9015 | F1: 0.7612 | Var: 0.

{'accuracy': 0.9052902800946542,
 'precision': 0.6506401733987608,
 'recall_tpr': 0.948437261419561,
 'f1_score': 0.7718092580025426,
 'fpr': 0.10347675006921044,
 'roc_auc': 0.9224802556751752,
 'test_loss': 1.5098989403713072,
 'tp': 80749,
 'tn': 375654,
 'fp': 43358,
 'fn': 4390,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '2;6;3',
 'client_variance': 0.00015169949024462095,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 39520512,
 'round_time_sec': 220.67381072044373}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "gaussian"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2 or sigma=None for labelflip
    sigma=5,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING GOSSIP - Attack: gaussian (20%)


2025-07-23 18:41:48.118327: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Round  1/30 | Acc: 0.1690 | F1: 0.2890 | Var: 0.0777 | Malicious: 3
Round  2/30 | Acc: 0.4763 | F1: 0.3706 | Var: 0.0431 | Malicious: 3
Round  3/30 | Acc: 0.4986 | F1: 0.3901 | Var: 0.0315 | Malicious: 3
Round  4/30 | Acc: 0.8479 | F1: 0.5687 | Var: 0.0298 | Malicious: 3
Round  5/30 | Acc: 0.9237 | F1: 0.7160 | Var: 0.0265 | Malicious: 3
Round  6/30 | Acc: 0.8993 | F1: 0.6289 | Var: 0.0273 | Malicious: 3
Round  7/30 | Acc: 0.9040 | F1: 0.7137 | Var: 0.0533 | Malicious: 3
Round  8/30 | Acc: 0.8736 | F1: 0.6898 | Var: 0.0192 | Malicious: 3
Round  9/30 | Acc: 0.8837 | F1: 0.7140 | Var: 0.0109 | Malicious: 3
Round 10/30 | Acc: 0.9487 | F1: 0.8260 | Var: 0.0329 | Malicious: 3
Round 11/30 | Acc: 0.9245 | F1: 0.7188 | Var: 0.0027 | Malicious: 3
Round 12/30 | Acc: 0.7795 | F1: 0.5096 | Var: 0.0632 | Malicious: 3
Round 13/30 | Acc: 0.8086 | F1: 0.5901 | Var: 0.0409 | Malicious: 3
Round 14/30 | Acc: 0.7864 | F1: 0.6017 | Var: 0.0563 | Malicious: 3
Round 15/30 | Acc: 0.7693 | F1: 0.5546 | Var: 0.

{'accuracy': 0.8933811496952302,
 'precision': 0.6225355851740804,
 'recall_tpr': 0.9364685984096595,
 'f1_score': 0.7478941147777798,
 'fpr': 0.11537378404437104,
 'roc_auc': 0.911523993225199,
 'test_loss': 1.6954301227019055,
 'tp': 79730,
 'tn': 370669,
 'fp': 48343,
 'fn': 5409,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'gaussian',
 'attack_pct': 0.2,
 'sigma': 5,
 'malicious_clients': '9;8;5',
 'client_variance': 0.01889735651663509,
 'comm_bytes_round': 3567824,
 'total_comm_bytes': 103604120,
 'round_time_sec': 249.23414587974548}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIoMT2024 dataset...
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING FEDAVG - Attack: none (0%)


I0000 00:00:1753051809.828086      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753051813.707539      98 service.cc:148] XLA service 0x7c703400b4f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753051813.708450      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753051814.010563      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753051815.602211      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8759 | F1: 0.0334 | Var: 0.1153 | Malicious: 0
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0002 | Malicious: 0
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.8746936274509803,
 'precision': 0.0,
 'recall_tpr': 0.0,
 'f1_score': 0.0,
 'fpr': 0.0,
 'roc_auc': 0.8224193817734939,
 'test_loss': 0.4686872476779285,
 'tp': 0,
 'tn': 2855,
 'fp': 0,
 'fn': 409,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 1.232595164407831e-32,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 187.32710313796997}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING FEDAVG - Attack: labelflip (40%)


I0000 00:00:1753196115.083565   20955 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753196117.740614   21004 service.cc:148] XLA service 0x7af6e000b900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753196117.740679   21004 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753196117.975939   21004 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753196119.380342   21004 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.0485 | F1: 0.0918 | Var: 0.1545 | Malicious: 6
Round  2/30 | Acc: 0.9534 | F1: 0.8424 | Var: 0.1747 | Malicious: 6
Round  3/30 | Acc: 0.9740 | F1: 0.9175 | Var: 0.1729 | Malicious: 6
Round  4/30 | Acc: 0.9518 | F1: 0.8704 | Var: 0.1433 | Malicious: 6
Round  5/30 | Acc: 0.0666 | F1: 0.1247 | Var: 0.1908 | Malicious: 6
Round  6/30 | Acc: 0.0296 | F1: 0.0462 | Var: 0.1864 | Malicious: 6
Round  7/30 | Acc: 0.9687 | F1: 0.8979 | Var: 0.1824 | Malicious: 6
Round  8/30 | Acc: 0.9699 | F1: 0.9072 | Var: 0.1773 | Malicious: 6
Round  9/30 | Acc: 0.9757 | F1: 0.9229 | Var: 0.1798 | Malicious: 6
Round 10/30 | Acc: 0.9731 | F1: 0.9137 | Var: 0.1501 | Malicious: 6
Round 11/30 | Acc: 0.9653 | F1: 0.8968 | Var: 0.1306 | Malicious: 6
Round 12/30 | Acc: 0.0333 | F1: 0.0641 | Var: 0.1872 | Malicious: 6
Round 13/30 | Acc: 0.0344 | F1: 0.0434 | Var: 0.1970 | Malicious: 6
Round 14/30 | Acc: 0.9698 | F1: 0.9054 | Var: 0.1960 | Malicious: 6
Round 15/30 | Acc: 0.9715 | F1: 0.9103 | Var: 0.

{'accuracy': 0.9743370537795224,
 'precision': 0.9977662874870734,
 'recall_tpr': 0.8499395106825309,
 'f1_score': 0.9179394154658006,
 'fpr': 0.00038662377211153855,
 'roc_auc': 0.9980781761311215,
 'test_loss': 0.05238734428699871,
 'tp': 72363,
 'tn': 418850,
 'fp': 162,
 'fn': 12776,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'labelflip',
 'attack_pct': 0.4,
 'sigma': None,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.20164317843789012,
 'comm_bytes_round': 2195584,
 'total_comm_bytes': 64769728,
 'round_time_sec': 224.85477423667908}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING FCWTL - Attack: labelflip (40%)


I0000 00:00:1753260278.402498      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753260281.538514      95 service.cc:148] XLA service 0x7df884009740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753260281.538905      95 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753260281.820326      95 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753260283.305014      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.9422 | F1: 0.7943 | Var: 0.0000 | Malicious: 6
Round  2/30 | Acc: 0.9533 | F1: 0.8398 | Var: 0.0000 | Malicious: 6
Round  3/30 | Acc: 0.8737 | F1: 0.4721 | Var: 0.0000 | Malicious: 6
Round  4/30 | Acc: 0.9553 | F1: 0.8478 | Var: 0.0000 | Malicious: 6
Round  5/30 | Acc: 0.6491 | F1: 0.3123 | Var: 0.0000 | Malicious: 6
Round  6/30 | Acc: 0.9067 | F1: 0.7833 | Var: 0.0000 | Malicious: 6
Round  7/30 | Acc: 0.0718 | F1: 0.0011 | Var: 0.0000 | Malicious: 6
Round  8/30 | Acc: 0.7118 | F1: 0.0108 | Var: 0.0000 | Malicious: 6
Round  9/30 | Acc: 0.5011 | F1: 0.0009 | Var: 0.0000 | Malicious: 6
Round 10/30 | Acc: 0.8634 | F1: 0.7080 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.5665 | F1: 0.4346 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.9305 | F1: 0.8291 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.8837 | F1: 0.4750 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.0413 | F1: 0.0571 | Var: 0.0000 | Malicious: 6
Round 15/30 | Acc: 0.4796 | F1: 0.0491 | Var: 0.

{'accuracy': 0.6557122766790109,
 'precision': 0.3287954466256243,
 'recall_tpr': 0.9974042448231715,
 'f1_score': 0.4945589661307653,
 'fpr': 0.41371607495728047,
 'roc_auc': 0.9420852433804253,
 'test_loss': 1.3152691871939646,
 'tp': 84918,
 'tn': 245660,
 'fp': 173352,
 'fn': 221,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'labelflip',
 'attack_pct': 0.4,
 'sigma': None,
 'malicious_clients': '10;12;9;4;14;3',
 'client_variance': 0.0,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 32933760,
 'round_time_sec': 145.76854157447815}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING RING - Attack: labelflip (20%)


I0000 00:00:1753264121.197943   11733 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753264123.807365   11785 service.cc:148] XLA service 0x783dc800a680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753264123.807408   11785 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753264124.021721   11785 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753264125.394637   11785 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.9478 | F1: 0.8174 | Var: 0.1211 | Malicious: 3
Round  2/30 | Acc: 0.9511 | F1: 0.8311 | Var: 0.0938 | Malicious: 3
Round  3/30 | Acc: 0.9560 | F1: 0.8506 | Var: 0.0702 | Malicious: 3
Round  4/30 | Acc: 0.9147 | F1: 0.7978 | Var: 0.0963 | Malicious: 3
Round  5/30 | Acc: 0.9263 | F1: 0.8205 | Var: 0.0958 | Malicious: 3
Round  6/30 | Acc: 0.9621 | F1: 0.8743 | Var: 0.0969 | Malicious: 3
Round  7/30 | Acc: 0.9439 | F1: 0.8011 | Var: 0.0846 | Malicious: 3
Round  8/30 | Acc: 0.5077 | F1: 0.1412 | Var: 0.0904 | Malicious: 3
Round  9/30 | Acc: 0.9645 | F1: 0.8829 | Var: 0.0962 | Malicious: 3
Round 10/30 | Acc: 0.9618 | F1: 0.8729 | Var: 0.1050 | Malicious: 3
Round 11/30 | Acc: 0.9325 | F1: 0.8331 | Var: 0.0813 | Malicious: 3
Round 12/30 | Acc: 0.9686 | F1: 0.8982 | Var: 0.1051 | Malicious: 3
Round 13/30 | Acc: 0.9646 | F1: 0.8831 | Var: 0.0802 | Malicious: 3
Round 14/30 | Acc: 0.7158 | F1: 0.0603 | Var: 0.0686 | Malicious: 3
Round 15/30 | Acc: 0.9601 | F1: 0.8662 | Var: 0.

{'accuracy': 0.9726570015729414,
 'precision': 0.9962996967420639,
 'recall_tpr': 0.8412126052690306,
 'f1_score': 0.9122114313007482,
 'fpr': 0.0006348266875411683,
 'roc_auc': 0.997116799933408,
 'test_loss': 0.0824496027753525,
 'tp': 71620,
 'tn': 418746,
 'fp': 266,
 'fn': 13519,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'labelflip',
 'attack_pct': 0.2,
 'sigma': None,
 'malicious_clients': '2;6;3',
 'client_variance': 0.13108800886105437,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 32933760,
 'round_time_sec': 220.2628083229065}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FEDAVG - Attack: labelflip (40%)


I0000 00:00:1753278060.028581      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753278063.684118      99 service.cc:148] XLA service 0x7cfad000c8a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753278063.684840      99 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753278063.989464      99 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753278065.485332      99 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8750 | F1: 0.0286 | Var: 0.1240 | Malicious: 6
Round  2/30 | Acc: 0.8750 | F1: 0.0049 | Var: 0.0439 | Malicious: 6
Round  3/30 | Acc: 0.8744 | F1: 0.0049 | Var: 0.0645 | Malicious: 6
Round  4/30 | Acc: 0.1253 | F1: 0.2227 | Var: 0.1090 | Malicious: 6
Round  5/30 | Acc: 0.7659 | F1: 0.2236 | Var: 0.0894 | Malicious: 6
Round  6/30 | Acc: 0.1253 | F1: 0.2227 | Var: 0.0943 | Malicious: 6
Round  7/30 | Acc: 0.8741 | F1: 0.0000 | Var: 0.1169 | Malicious: 6
Round  8/30 | Acc: 0.8741 | F1: 0.0000 | Var: 0.0652 | Malicious: 6
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0318 | Malicious: 6
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.8746936274509803,
 'precision': 0.0,
 'recall_tpr': 0.0,
 'f1_score': 0.0,
 'fpr': 0.0,
 'roc_auc': 0.8343839786930662,
 'test_loss': 0.3989776413860385,
 'tp': 0,
 'tn': 2855,
 'fp': 0,
 'fn': 409,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'labelflip',
 'attack_pct': 0.4,
 'sigma': None,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 2.1025567089583326e-07,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 183.09810829162598}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FCWTL - Attack: labelflip (20%)


I0000 00:00:1753295586.359178      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753295590.143998      99 service.cc:148] XLA service 0x7fa24400c380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753295590.144952      99 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753295590.436252      99 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753295591.957406      99 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  6/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  7/30 | Acc: 0.9344 | F1: 0.6503 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0000 | Malicious: 3
Round  9/30 | Acc: 0.9332 | F1: 0.6414 | Var: 0.0000 | Malicious: 3
Round 10/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.0000 | Malicious: 3
Round 11/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.6388 | F1: 0.3561 | Var: 0.0000 | Malicious: 3
Round 13/30 | Acc: 0.9372 | F1: 0.6730 | Var: 0.0000 | Malicious: 3
Round 14/30 | Acc: 0.9344 | F1: 0.6503 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.

{'accuracy': 0.9034926470588235,
 'precision': 0.6223958333333334,
 'recall_tpr': 0.5843520782396088,
 'f1_score': 0.6027742749054225,
 'fpr': 0.050788091068301226,
 'roc_auc': 0.8564633744256848,
 'test_loss': 0.2725033128350156,
 'tp': 239,
 'tn': 2710,
 'fp': 145,
 'fn': 170,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'labelflip',
 'attack_pct': 0.2,
 'sigma': None,
 'malicious_clients': '12;5;8',
 'client_variance': 0.0,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 117.613126039505}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4 #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING RING - Attack: labelflip (40%)


I0000 00:00:1753308157.607074      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753308161.256672      97 service.cc:148] XLA service 0x7b9f8c00a590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753308161.257413      97 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753308161.548343      97 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753308163.055174      97 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0543 | Malicious: 6
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round  4/30 | Acc: 0.1253 | F1: 0.2227 | Var: 0.1126 | Malicious: 6
Round  5/30 | Acc: 0.8747 | F1: 0.0285 | Var: 0.1127 | Malicious: 6
Round  6/30 | Acc: 0.1253 | F1: 0.2227 | Var: 0.0827 | Malicious: 6
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0345 | Malicious: 6
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.8765 | F1: 0.0289 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.8765 | F1: 0.0289 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.9105 | F1: 0.4511 | Var: 0.0002 | Malicious: 6
Round 15/30 | Acc: 0.8762 | F1: 0.0242 | Var: 0.

{'accuracy': 0.9332107843137255,
 'precision': 0.9798994974874372,
 'recall_tpr': 0.4767726161369193,
 'f1_score': 0.6414473684210527,
 'fpr': 0.0014010507880910684,
 'roc_auc': 0.8631727463078971,
 'test_loss': 0.2910808443900328,
 'tp': 195,
 'tn': 2851,
 'fp': 4,
 'fn': 214,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'labelflip',
 'attack_pct': 0.4,
 'sigma': None,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.000307755063490538,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 184.22616505622864}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIoMT2024 dataset...
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING FEDAVG - Attack: labelflip (20%)


I0000 00:00:1753128877.357880      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753128881.171981      98 service.cc:148] XLA service 0x78538800b1f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753128881.172777      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753128881.475500      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753128883.064265      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8753 | F1: 0.0378 | Var: 0.1129 | Malicious: 3
Round  2/30 | Acc: 0.8762 | F1: 0.0242 | Var: 0.0360 | Malicious: 3
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0351 | Malicious: 3
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  6/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.9237132352941176,
 'precision': 0.9761904761904762,
 'recall_tpr': 0.40097799511002447,
 'f1_score': 0.5684575389948008,
 'fpr': 0.0014010507880910684,
 'roc_auc': 0.8339326622105945,
 'test_loss': 0.26791620502161545,
 'tp': 164,
 'tn': 2851,
 'fp': 4,
 'fn': 245,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'labelflip',
 'attack_pct': 0.2,
 'sigma': None,
 'malicious_clients': '2;6;3',
 'client_variance': 0.00039066004261181607,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 192.98425197601318}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIoMT2024 dataset...
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING RING - Attack: labelflip (20%)


I0000 00:00:1753135907.108793   20955 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753135910.451724   21005 service.cc:148] XLA service 0x7a51f400b170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753135910.451783   21005 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753135910.671738   21005 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753135912.119870   21005 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0292 | Malicious: 3
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  6/30 | Acc: 0.8940 | F1: 0.2762 | Var: 0.0000 | Malicious: 3
Round  7/30 | Acc: 0.8820 | F1: 0.1149 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.9026 | F1: 0.3715 | Var: 0.0002 | Malicious: 3
Round  9/30 | Acc: 0.9176 | F1: 0.5171 | Var: 0.0004 | Malicious: 3
Round 10/30 | Acc: 0.9372 | F1: 0.6699 | Var: 0.0003 | Malicious: 3
Round 11/30 | Acc: 0.9344 | F1: 0.6656 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.0002 | Malicious: 3
Round 13/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.0006 | Malicious: 3
Round 14/30 | Acc: 0.9032 | F1: 0.5980 | Var: 0.0002 | Malicious: 3
Round 15/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.

{'accuracy': 0.2536764705882353,
 'precision': 0.14300810144417048,
 'recall_tpr': 0.9926650366748166,
 'f1_score': 0.25,
 'fpr': 0.8521891418563923,
 'roc_auc': 0.8294383379221458,
 'test_loss': 1.0935623186148489,
 'tp': 406,
 'tn': 422,
 'fp': 2433,
 'fn': 3,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'labelflip',
 'attack_pct': 0.2,
 'sigma': None,
 'malicious_clients': '2;6;3',
 'client_variance': 0.07596456624588835,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 189.65232229232788}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING RING - Attack: none (0%)


I0000 00:00:1753260322.727445      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753260326.337094      95 service.cc:148] XLA service 0x7ef124009530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753260326.338177      95 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753260326.687105      95 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753260328.411128      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.9562 | F1: 0.8514 | Var: 0.0156 | Malicious: 0
Round  2/30 | Acc: 0.9586 | F1: 0.8609 | Var: 0.0002 | Malicious: 0
Round  3/30 | Acc: 0.9632 | F1: 0.8783 | Var: 0.0002 | Malicious: 0
Round  4/30 | Acc: 0.9622 | F1: 0.8745 | Var: 0.0002 | Malicious: 0
Round  5/30 | Acc: 0.9678 | F1: 0.8954 | Var: 0.0002 | Malicious: 0
Round  6/30 | Acc: 0.9674 | F1: 0.8938 | Var: 0.0002 | Malicious: 0
Round  7/30 | Acc: 0.9668 | F1: 0.8917 | Var: 0.0001 | Malicious: 0
Round  8/30 | Acc: 0.9698 | F1: 0.9025 | Var: 0.0001 | Malicious: 0
Round  9/30 | Acc: 0.9668 | F1: 0.8913 | Var: 0.0001 | Malicious: 0
Round 10/30 | Acc: 0.9713 | F1: 0.9078 | Var: 0.0001 | Malicious: 0
Round 11/30 | Acc: 0.9712 | F1: 0.9073 | Var: 0.0001 | Malicious: 0
Round 12/30 | Acc: 0.9742 | F1: 0.9180 | Var: 0.0001 | Malicious: 0
Round 13/30 | Acc: 0.9764 | F1: 0.9253 | Var: 0.0001 | Malicious: 0
Round 14/30 | Acc: 0.9751 | F1: 0.9211 | Var: 0.0001 | Malicious: 0
Round 15/30 | Acc: 0.9735 | F1: 0.9154 | Var: 0.

{'accuracy': 0.9766934906407009,
 'precision': 0.9917844937345038,
 'recall_tpr': 0.8691903827857973,
 'f1_score': 0.9264494159770648,
 'fpr': 0.0014629652611381058,
 'roc_auc': 0.9978420436263409,
 'test_loss': 0.07268682391702604,
 'tp': 74002,
 'tn': 418399,
 'fp': 613,
 'fn': 11137,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 0.0002905136825959228,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 32933760,
 'round_time_sec': 242.84546852111816}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FEDAVG - Attack: none (0%)


I0000 00:00:1753266148.715964   21970 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753266152.361846   22024 service.cc:148] XLA service 0x7d013000cf50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753266152.361893   22024 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753266152.638277   22024 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753266154.309003   22024 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8759 | F1: 0.0334 | Var: 0.1153 | Malicious: 0
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0002 | Malicious: 0
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.8746936274509803,
 'precision': 0.0,
 'recall_tpr': 0.0,
 'f1_score': 0.0,
 'fpr': 0.0,
 'roc_auc': 0.8224193817734939,
 'test_loss': 0.4686872476779285,
 'tp': 0,
 'tn': 2855,
 'fp': 0,
 'fn': 409,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 1.232595164407831e-32,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 202.98815846443176}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FCWTL - Attack: none (0%)


I0000 00:00:1753270781.443562   42884 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753270785.084436   42937 service.cc:148] XLA service 0x7c026800b6f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753270785.084488   42937 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753270785.317518   42937 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753270786.967662   42937 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.8977 | F1: 0.3184 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.9347 | F1: 0.6525 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.9369 | F1: 0.6677 | Var: 0.0000 | Malicious: 0
Round  7/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0000 | Malicious: 0
Round  8/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0000 | Malicious: 0
Round  9/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0000 | Malicious: 0
Round 10/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0000 | Malicious: 0
Round 11/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0000 | Malicious: 0
Round 12/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0000 | Malicious: 0
Round 13/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.0000 | Malicious: 0
Round 14/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.0000 | Malicious: 0
Round 15/30 | Acc: 0.9378 | F1: 0.6742 | Var: 0.

{'accuracy': 0.9362745098039216,
 'precision': 0.935064935064935,
 'recall_tpr': 0.5281173594132029,
 'f1_score': 0.6749999999999999,
 'fpr': 0.005253940455341506,
 'roc_auc': 0.8488466594444611,
 'test_loss': 0.2836284720974684,
 'tp': 216,
 'tn': 2840,
 'fp': 15,
 'fn': 193,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 0.0,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 123.01237106323242}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4 #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING RING - Attack: labelflip (40%)


I0000 00:00:1753300244.338576   20961 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753300247.598478   21015 service.cc:148] XLA service 0x7bf89000b7c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753300247.598543   21015 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753300247.814141   21015 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753300249.221051   21015 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0543 | Malicious: 6
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round  4/30 | Acc: 0.1253 | F1: 0.2227 | Var: 0.1126 | Malicious: 6
Round  5/30 | Acc: 0.8747 | F1: 0.0285 | Var: 0.1127 | Malicious: 6
Round  6/30 | Acc: 0.1253 | F1: 0.2227 | Var: 0.0827 | Malicious: 6
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0345 | Malicious: 6
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 11/30 | Acc: 0.8765 | F1: 0.0289 | Var: 0.0000 | Malicious: 6
Round 12/30 | Acc: 0.8765 | F1: 0.0289 | Var: 0.0000 | Malicious: 6
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 6
Round 14/30 | Acc: 0.9105 | F1: 0.4511 | Var: 0.0002 | Malicious: 6
Round 15/30 | Acc: 0.8762 | F1: 0.0242 | Var: 0.

{'accuracy': 0.9332107843137255,
 'precision': 0.9798994974874372,
 'recall_tpr': 0.4767726161369193,
 'f1_score': 0.6414473684210527,
 'fpr': 0.0014010507880910684,
 'roc_auc': 0.8631727463078971,
 'test_loss': 0.2910808443900328,
 'tp': 195,
 'tn': 2851,
 'fp': 4,
 'fn': 214,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'labelflip',
 'attack_pct': 0.4,
 'sigma': None,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.000307755063490538,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 185.10165524482727}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fcwtl"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING FCWTL - Attack: none (0%)


I0000 00:00:1753260227.206752      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753260230.510072      97 service.cc:148] XLA service 0x7db51c00b1c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753260230.510572      97 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753260230.804898      97 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753260232.336262      97 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.9562 | F1: 0.8514 | Var: 0.0000 | Malicious: 0
Round  2/30 | Acc: 0.9573 | F1: 0.8558 | Var: 0.0000 | Malicious: 0
Round  3/30 | Acc: 0.9626 | F1: 0.8761 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.9643 | F1: 0.8824 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.9642 | F1: 0.8821 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.9657 | F1: 0.8875 | Var: 0.0000 | Malicious: 0
Round  7/30 | Acc: 0.9690 | F1: 0.8995 | Var: 0.0000 | Malicious: 0
Round  8/30 | Acc: 0.9682 | F1: 0.8968 | Var: 0.0000 | Malicious: 0
Round  9/30 | Acc: 0.9705 | F1: 0.9049 | Var: 0.0000 | Malicious: 0
Round 10/30 | Acc: 0.9724 | F1: 0.9116 | Var: 0.0000 | Malicious: 0
Round 11/30 | Acc: 0.9736 | F1: 0.9157 | Var: 0.0000 | Malicious: 0
Round 12/30 | Acc: 0.9756 | F1: 0.9229 | Var: 0.0000 | Malicious: 0
Round 13/30 | Acc: 0.9753 | F1: 0.9215 | Var: 0.0000 | Malicious: 0
Round 14/30 | Acc: 0.9741 | F1: 0.9174 | Var: 0.0000 | Malicious: 0
Round 15/30 | Acc: 0.9765 | F1: 0.9255 | Var: 0.

{'accuracy': 0.9768085355379639,
 'precision': 0.9956004804383325,
 'recall_tpr': 0.8665006636206674,
 'f1_score': 0.9265753149373893,
 'fpr': 0.0007780206772121085,
 'roc_auc': 0.9980487673803434,
 'test_loss': 0.06219660887998003,
 'tp': 73773,
 'tn': 418686,
 'fp': 326,
 'fn': 11366,
 'round': 30,
 'topology': 'fcwtl',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 0.0,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 32933760,
 'round_time_sec': 145.3661482334137}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.4               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING RING - Attack: labelflip (40%)


I0000 00:00:1753264166.487701   11730 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753264169.195829   11781 service.cc:148] XLA service 0x7b766800a490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753264169.195868   11781 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753264169.410665   11781 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753264170.757918   11781 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.9422 | F1: 0.7943 | Var: 0.1736 | Malicious: 6
Round  2/30 | Acc: 0.6935 | F1: 0.5184 | Var: 0.1394 | Malicious: 6
Round  3/30 | Acc: 0.9108 | F1: 0.7881 | Var: 0.1404 | Malicious: 6
Round  4/30 | Acc: 0.9327 | F1: 0.8333 | Var: 0.1452 | Malicious: 6
Round  5/30 | Acc: 0.9535 | F1: 0.8411 | Var: 0.2004 | Malicious: 6
Round  6/30 | Acc: 0.7312 | F1: 0.2993 | Var: 0.1555 | Malicious: 6
Round  7/30 | Acc: 0.8311 | F1: 0.0000 | Var: 0.1506 | Malicious: 6
Round  8/30 | Acc: 0.8284 | F1: 0.0055 | Var: 0.1549 | Malicious: 6
Round  9/30 | Acc: 0.0669 | F1: 0.1245 | Var: 0.1873 | Malicious: 6
Round 10/30 | Acc: 0.9028 | F1: 0.7763 | Var: 0.1558 | Malicious: 6
Round 11/30 | Acc: 0.6762 | F1: 0.5101 | Var: 0.1305 | Malicious: 6
Round 12/30 | Acc: 0.9600 | F1: 0.8660 | Var: 0.1852 | Malicious: 6
Round 13/30 | Acc: 0.0401 | F1: 0.0764 | Var: 0.1881 | Malicious: 6
Round 14/30 | Acc: 0.8215 | F1: 0.0960 | Var: 0.1791 | Malicious: 6
Round 15/30 | Acc: 0.9278 | F1: 0.8236 | Var: 0.

{'accuracy': 0.742710021402318,
 'precision': 0.007578435704816615,
 'recall_tpr': 0.0040287059984261036,
 'f1_score': 0.00526077653969739,
 'fpr': 0.10719740723416035,
 'roc_auc': 0.19688587416272932,
 'test_loss': 0.9208823683367483,
 'tp': 343,
 'tn': 374095,
 'fp': 44917,
 'fn': 84796,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'labelflip',
 'attack_pct': 0.4,
 'sigma': None,
 'malicious_clients': '3;0;8;7;13;5',
 'client_variance': 0.1577453849744642,
 'comm_bytes_round': 1097792,
 'total_comm_bytes': 32933760,
 'round_time_sec': 223.60466384887695}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "fedavg"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING FEDAVG - Attack: labelflip (20%)


I0000 00:00:1753278087.680701      76 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753278091.979251     127 service.cc:148] XLA service 0x7cfb0008d070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753278091.980315     127 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753278092.295253     127 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753278093.895722     127 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8753 | F1: 0.0378 | Var: 0.1129 | Malicious: 3
Round  2/30 | Acc: 0.8762 | F1: 0.0242 | Var: 0.0360 | Malicious: 3
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0351 | Malicious: 3
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  5/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  6/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  7/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.9234068627450981,
 'precision': 0.9760479041916168,
 'recall_tpr': 0.39853300733496333,
 'f1_score': 0.5659722222222222,
 'fpr': 0.0014010507880910684,
 'roc_auc': 0.8337815097264268,
 'test_loss': 0.2679224349681158,
 'tp': 163,
 'tn': 2851,
 'fp': 4,
 'fn': 246,
 'round': 30,
 'topology': 'fedavg',
 'attack_type': 'labelflip',
 'attack_pct': 0.2,
 'sigma': None,
 'malicious_clients': '2;6;3',
 'client_variance': 0.0003939899250822334,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 189.75809931755066}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "ciciomt2024"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "ring"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIoMT2024 dataset...
Original shape: (16318, 45)
Removed 6 constant columns
Final shape: (16318, 31), Target distribution: [14272  2046]
Numeric columns: 30, Categorical columns: 1
Preprocessed - Train: (13054, 13082), Test: (3264, 13082)

RUNNING RING - Attack: none (0%)


I0000 00:00:1753301668.794682   14465 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1753301671.978001   14520 service.cc:148] XLA service 0x79f3c400c390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753301671.978039   14520 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753301672.187909   14520 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753301673.554037   14520 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Round  1/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  2/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  3/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  4/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 0
Round  5/30 | Acc: 0.8928 | F1: 0.2585 | Var: 0.0000 | Malicious: 0
Round  6/30 | Acc: 0.9357 | F1: 0.6591 | Var: 0.0004 | Malicious: 0
Round  7/30 | Acc: 0.9372 | F1: 0.6699 | Var: 0.0003 | Malicious: 0
Round  8/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round  9/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 10/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 11/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0002 | Malicious: 0
Round 12/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 13/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 14/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.0001 | Malicious: 0
Round 15/30 | Acc: 0.9375 | F1: 0.6720 | Var: 0.

{'accuracy': 0.9356617647058824,
 'precision': 0.9128630705394191,
 'recall_tpr': 0.5378973105134475,
 'f1_score': 0.676923076923077,
 'fpr': 0.007355516637478109,
 'roc_auc': 0.8569116935501137,
 'test_loss': 0.2712423780737428,
 'tp': 220,
 'tn': 2834,
 'fp': 21,
 'fn': 189,
 'round': 30,
 'topology': 'ring',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 0.00014769793402110278,
 'comm_bytes_round': 107839552,
 'total_comm_bytes': 3235186560,
 'round_time_sec': 183.7105085849762}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "wustl2020"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "labelflip"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.2               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading WUSTL‑EHMS‑2020 from /kaggle/input/wustlehms2020/wustl-ehms-2020.csv
Numeric 30 | Categorical 1
Pre‑processed WUSTL‑2020 – Train (13054, 13082) | Test (3264, 13082)

RUNNING GOSSIP - Attack: labelflip (20%)


2025-07-24 11:14:40.566400: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Round  1/30 | Acc: 0.8729 | F1: 0.0547 | Var: 0.1068 | Malicious: 3
Round  2/30 | Acc: 0.8759 | F1: 0.0288 | Var: 0.0355 | Malicious: 3
Round  3/30 | Acc: 0.8768 | F1: 0.0337 | Var: 0.0372 | Malicious: 3
Round  4/30 | Acc: 0.8762 | F1: 0.1513 | Var: 0.0706 | Malicious: 3
Round  5/30 | Acc: 0.8765 | F1: 0.0289 | Var: 0.0700 | Malicious: 3
Round  6/30 | Acc: 0.8753 | F1: 0.0097 | Var: 0.0000 | Malicious: 3
Round  7/30 | Acc: 0.8753 | F1: 0.0097 | Var: 0.0000 | Malicious: 3
Round  8/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round  9/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 10/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 11/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 12/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 13/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 14/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.0000 | Malicious: 3
Round 15/30 | Acc: 0.8747 | F1: 0.0000 | Var: 0.

{'accuracy': 0.8746936274509803,
 'precision': 0.0,
 'recall_tpr': 0.0,
 'f1_score': 0.0,
 'fpr': 0.0,
 'roc_auc': 0.8292653475436652,
 'test_loss': 0.3226272437405476,
 'tp': 0,
 'tn': 2855,
 'fp': 0,
 'fn': 409,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'labelflip',
 'attack_pct': 0.2,
 'sigma': None,
 'malicious_clients': '9;8;5',
 'client_variance': 7.09195715323418e-08,
 'comm_bytes_round': 202199160,
 'total_comm_bytes': 6065974800,
 'round_time_sec': 178.16266512870789}

In [ ]:
# ---------------------- choose experiment ----------------------
dataset   = "cicids2017"      # {"ciciomt2024", "cicids2017", "wustl2020"}
topology  = "gossip"           # {"fedavg", "fcwtl", "ring", "gossip"}
attack    = "none"        # {"none", "labelflip", "gaussian"}
mal_pct   = 0.0               #  0.2 -0.4

# ---------------------- kick off run ---------------------------
run_federated_learning(
    topology     = topology,
    dataset      = dataset,
    attack_type  = attack,
    attack_pct   = mal_pct,
    # sigma is only needed for Gaussian poisoning sigma=2
    sigma=None,
    n_clients    = 15,
    rounds       = 30,
    local_epochs = 1,
    batch_size   = 256,
    dir_alpha    = 0.30,
    output_dir   = f"{dataset}_{topology}_{attack}_{int(mal_pct*100)}"
)

Loading CICIDS‑2017 from /kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
Pre‑processed CICIDS – Train (2016600, 52) | Test (504151, 52)

RUNNING GOSSIP - Attack: none (0%)


2025-07-24 14:13:07.866708: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Round  1/30 | Acc: 0.9277 | F1: 0.7279 | Var: 0.0164 | Malicious: 0
Round  2/30 | Acc: 0.9634 | F1: 0.8823 | Var: 0.0002 | Malicious: 0
Round  3/30 | Acc: 0.9676 | F1: 0.8971 | Var: 0.0002 | Malicious: 0
Round  4/30 | Acc: 0.9638 | F1: 0.8800 | Var: 0.0002 | Malicious: 0
Round  5/30 | Acc: 0.9660 | F1: 0.8881 | Var: 0.0001 | Malicious: 0
Round  6/30 | Acc: 0.9757 | F1: 0.9235 | Var: 0.0002 | Malicious: 0
Round  7/30 | Acc: 0.9772 | F1: 0.9286 | Var: 0.0001 | Malicious: 0
Round  8/30 | Acc: 0.9776 | F1: 0.9300 | Var: 0.0001 | Malicious: 0
Round  9/30 | Acc: 0.9779 | F1: 0.9319 | Var: 0.0001 | Malicious: 0
Round 10/30 | Acc: 0.9730 | F1: 0.9134 | Var: 0.0000 | Malicious: 0
Round 11/30 | Acc: 0.9737 | F1: 0.9158 | Var: 0.0001 | Malicious: 0
Round 12/30 | Acc: 0.9764 | F1: 0.9257 | Var: 0.0002 | Malicious: 0
Round 13/30 | Acc: 0.9741 | F1: 0.9170 | Var: 0.0002 | Malicious: 0
Round 14/30 | Acc: 0.9751 | F1: 0.9206 | Var: 0.0001 | Malicious: 0
Round 15/30 | Acc: 0.9743 | F1: 0.9179 | Var: 0.

{'accuracy': 0.9775146731832328,
 'precision': 0.981950448626693,
 'recall_tpr': 0.8830853075558792,
 'f1_score': 0.9298974682448394,
 'fpr': 0.003298234895420656,
 'roc_auc': 0.9982077464334715,
 'test_loss': 0.045623810757398194,
 'tp': 75185,
 'tn': 417630,
 'fp': 1382,
 'fn': 9954,
 'round': 30,
 'topology': 'gossip',
 'attack_type': 'none',
 'attack_pct': 0.0,
 'sigma': None,
 'malicious_clients': '',
 'client_variance': 4.4801016256494327e-05,
 'comm_bytes_round': 3156152,
 'total_comm_bytes': 93586768,
 'round_time_sec': 214.94580507278442}